In [ ]:
# %% [Instalación completa]
!apt-get update
!apt-get install -y poppler-utils tesseract-ocr tesseract-ocr-spa

!pip install --upgrade pip
!pip install pykeen pdf2image pytesseract Pillow nltk networkx matplotlib transformers torch huggingface-hub  pymupdf unidecode spacy owlready2
!pip install langdetect
# Descarga recursos NLTK
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')  # Útil para análisis adicional

!python -m spacy download es_core_news_lg

In [ ]:
!pip install mistralai

# Procesamiento del Texto

In [ ]:
STOPWORDS = set(
    [
        "introducción",
        "método",
        "métodos",
        "resultado",
        "resultados",
        "discusión",
        "conclusión",
        "figura",
        "tabla",
        "referencia",
        "estudio",
        "análisis",
        "datos",
        "artículo",
        "sección",
        "mostrado",
        "usando",
        "usado",
        "basado",
        "encontrado",
        "también",
        "sin embargo",
        "aunque",
        "año",
        "años",
        "tiempo",
        "alto",
        "bajo",
        "valor",
        "caso",
        "grupo",
        "et",
        "al",
        "probabilidad",
        "momento",
        "situaciones",
        "descubrir",
        "mantiene",
        "significaba",
        "quizás",
        "debido",
        "uso",
        "hacer",
        "obtener",
        "puede",
        "podría",
        "listado",
        "conferencias",
        "antecedentes",
        "significancia",
        "derechos de autor",
        "autor",
        "fig",
        "ec",
        "vol",
    ]
)

BLACKLIST = {
    "fig", "figura", "figure", "tabla", "table", "cuadro", "doi", "issn",
    "url", "http", "www", "et", "al", "vol", "no", "pág", "pag", "ed",
    "estudio", "análisis", "datos", "método", "resultado", "conclusión" # Palabras genéricas
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from idna import decode
import pdf2image
import fitz
import pytesseract
from unidecode import unidecode
import nltk
import hashlib
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
from pathlib import Path
import string
import spacy
import os
import re

try:
    nlp = spacy.load("es_core_news_lg")
    print("✅ Modelo spaCy 'es_core_news_lg' cargado correctamente.")
except OSError:
    print("⚠️ Error cargando spaCy. Asegúrate de haber ejecutado la celda de instalación.")


nltk.download("punkt")
nltk.download("stopwords")


ruta_local = os.path.expanduser("./model")
# Configurar la ruta de la carpeta
carpeta_txts = "/content/drive/MyDrive/txts"  # Cambiado a Google Drive
carpeta_pdfs = Path("/content/drive/MyDrive/corpus")
carpeta_pdfs = Path(carpeta_pdfs)
article_txt = ""
all_articles_text = []

# Asegurarse de que la carpeta existe
if not carpeta_pdfs.exists():
    carpeta_pdfs.mkdir(parents=True, exist_ok=True)
    print(f"Carpeta '{carpeta_pdfs}' creada. Coloca tus PDFs allí.")
else:
    # Procesar todos los PDFs en la carpeta
    # Initialize article_txt to ensure it's set for each iteration or if no PDFs are processed
    article_txt = ""  # Esta variable se usará globalmente, pero la reiniciaremos por PDF

    for pdf_file in carpeta_pdfs.glob("*.pdf"):
        print(f"\nProcesando: {pdf_file.name}")

        article_txt = ""  # Variable local para cada PDF

        try:
            # Construir nombre del archivo txt
            pdf_file_name = pdf_file.with_suffix(".txt").name
            file_name = Path(carpeta_txts) / pdf_file_name # FIX: Changed './txts' to 'carpeta_txts'

            print(file_name)

            # Si ya existe el .txt, cargarlo
            if file_name.exists():
                print(f"  El archivo {file_name.name} ya existe. Cargando texto existente...")
                with open(file_name, "r", encoding="utf-8") as f:
                    article_txt = f.read()
            else:
                # --- Extracción nueva del PDF ---
                doc = fitz.open(pdf_file)
                full_text_pages = []

                for page in doc:
                    blocks = page.get_text("blocks", sort=True)
                    page_text_parts = []
                    for block in blocks:
                        txt = block[4]
                        if not txt.strip() or len(txt) < 5:
                            continue
                        txt = re.sub(r'([a-záéíóúñ])([A-ZÁÉÍÓÚÑ])', r'\1 \2', txt)
                        page_text_parts.append(txt)

                    full_text_pages.append("\n".join(page_text_parts))

                article_txt = "\n".join(full_text_pages)
                article_txt = re.sub(r'\s+', ' ', article_txt).strip()

                # Guardar el texto extraído
                file_name.parent.mkdir(parents=True, exist_ok=True)
                with open(file_name, "w", encoding="utf-8") as f:
                    f.write(article_txt)

                print(f"  Texto extraído y guardado en {file_name.name}")

            # Agregar a la lista de todos los textos
            if article_txt.strip():
                print(article_txt)  # Solo si no está vacío
                all_articles_text.append(article_txt)

        except Exception as e:
            print(f"  Error procesando {pdf_file.name}: {e}")

# Combinar todos los textos
combined_text = "\n\n".join(all_articles_text)

In [ ]:
from langdetect import detect, LangDetectException

def is_spanish(text):
    try:
        # Si el texto es muy corto, langdetect falla o es impreciso
        if len(text.split()) < 4:
            return False
        return detect(text) == 'es'
    except LangDetectException:
        return False

# Mejora en la limpieza de texto para unir palabras partidas por guiones
def clean_text(text):
    text = re.split(r'\n\s*(Referencia|Bibliograf|Bibliography|References)', text, flags=re.IGNORECASE)[0]
    text = re.sub(r'(\w)-\n(\w)', r'\1\2', text)

    lines = text.split('\n')
    clean_lines = []
    for line in lines:
        # Filtra líneas que sean solo números o muy cortas (basura de paginación)
        if len(line.strip()) < 3 or line.strip().isdigit():
            continue
        clean_lines.append(line.strip())

    text = " ".join(clean_lines)

    # 4. Limpieza estándar
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print(combined_text)
ctext = clean_text(combined_text)
sentences = nltk.tokenize.sent_tokenize(ctext, language="spanish")
print(sentences)
print(ctext)

# NER

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

# Reemplazo de spaCy: Cargar pipeline de Hugging Face para NER biomédico en español
model_name = "HUMADEX/spanish_medical_ner"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model = AutoModelForTokenClassification.from_pretrained(model_name)

    # 2. Pasar el tokenizador explícito al pipeline
    ner_pipeline = pipeline(
        "ner",
        model=model,
        tokenizer=tokenizer,
        aggregation_strategy="max"
    )
except Exception as e:
    print(f"Error cargando modelo: {e}")
    raise

In [ ]:
entity_list = []

ENGLISH_STOPWORDS = {"the", "of", "and", "in", "to", "with", "a", "for", "study", "fistulae", "skull", "base", "flap", "nasoseptal"}
STOPWORDS.update(ENGLISH_STOPWORDS)

entity_list = []

# Tokenizar oraciones
sentences = nltk.tokenize.sent_tokenize(ctext, language="spanish")

for s in sentences:
    # 1. FILTRO DE IDIOMA: Si la oración no es español, sáltala completamente
    if not is_spanish(s):
        continue

    if len(s) < 20: continue

    # Ejecutar NER
    results = ner_pipeline(s)
    doc = nlp(s)

    denotations = []
    seen_lemmas = set() # Para evitar duplicados en la misma oración, pero sin romper orden interno

    for ent in results:
        if ent["score"] < 0.60: continue

        # Obtener span de spaCy para lematización precisa
        span = doc.char_span(ent["start"], ent["end"], alignment_mode="contract")

        if not span:
            # Fallback simple si spaCy no alinea: usar la palabra cruda
            lemma = ent["word"].strip().lower()
        else:
            # --- CORRECCIÓN CRÍTICA AQUÍ ---
            # NO usar set(). Usar lista para mantener orden: "mucosa nasal" != "nasal mucosa"
            lemma_parts = [t.lemma_.lower() for t in span]
            lemma = " ".join(lemma_parts)
            lemma = unidecode(lemma) # Quitar acentos para normalizar grafo

        # Limpieza final del lema
        lemma = re.sub(r'[^\w\s]', '', lemma).strip() # Quitar puntuación

        # Filtros de calidad del nodo
        if (len(lemma) < 4 or
            lemma in STOPWORDS or
            lemma in BLACKLIST or
            lemma.isdigit() or
            any(w in ENGLISH_STOPWORDS for w in lemma.split())): # Si contiene palabras en inglés
            continue

        label = ent["entity_group"]

        # Evitar agregar el mismo nodo dos veces para la misma oración
        if lemma not in seen_lemmas:
            denotations.append({
                "obj": label,
                "span": {"begin": ent["start"], "end": ent["end"]},
                "lemma": lemma, # Ahora es "cirugia endoscopica", no "endoscopica cirugia"
                "original": ent["word"]
            })
            seen_lemmas.add(lemma)

    if denotations:
        entity_list.append({"text": s, "denotations": denotations})

print(entity_list)

In [ ]:
parsed_entities = []
for entities_in_sentence in entity_list:
    e = []
    # If there are not entities in the text
    if not entities_in_sentence.get("denotations"):
        parsed_entities.append(
            {
                "text": entities_in_sentence["text"],
                "text_sha256": hashlib.sha256(
                    entities_in_sentence["text"].encode("utf-8")
                ).hexdigest(),
            }
        )
        continue
    for entity_denotation in entities_in_sentence["denotations"]:
        # The 'denotations' created above do not have an 'id' list.
        # The 'lemma' field is intended to be the unique identifier for the graph node.
        entity_id = entity_denotation["lemma"] # Use lemma as the ID
        other_ids = [] # No other IDs are being generated in this pipeline currently.
        entity_type = entity_denotation["obj"]

        # Use the already normalized lemma from the denotation for the entity name in the graph
        processed_lemma_for_graph = entity_denotation["lemma"]

        # Apply final filtering steps to this processed_lemma
        if processed_lemma_for_graph in STOPWORDS or len(processed_lemma_for_graph) < 3 or len(processed_lemma_for_graph) > 60 or processed_lemma_for_graph.isdigit() or not re.search(r'[a-záéíóúñ]', processed_lemma_for_graph, re.I):
            continue

        e.append(
            {
                "entity_id": entity_id, # This is the normalized lemma
                "other_ids": other_ids, # This will be empty
                "entity_type": entity_type,
                "entity": processed_lemma_for_graph, # Use the consistent lemma for the entity name in the graph
                "start": entity_denotation["span"]["begin"], # Add start index
                "end": entity_denotation["span"]["end"]    # Add end index
            }
        )

    parsed_entities.append(
        {
            "entities": e,
            "text": entities_in_sentence["text"],
            "text_sha256": hashlib.sha256(entities_in_sentence["text"].encode("utf-8")).hexdigest(),
        }
    )


# Construccion del Grafo de Conocimiento

In [ ]:
!pip install numpy
import numpy as np

def check_dependency(token1, token2):
    """
    Retorna True si hay una conexión gramatical fuerte entre dos tokens.
    """
    # 1. Uno es ancestro del otro (ej: "tumor" -> "cerebral")
    if token1 in token2.ancestors or token2 in token1.ancestors:
        return True

    # 2. Comparten el mismo padre inmediato y ese padre es VERBO o AUXILIAR
    # Ej: "La resección (Sujeto) ELIMINÓ (Verbo) el tumor (Objeto)"
    if token1.head == token2.head and token1.head.pos_ in ["VERB", "AUX"]:
        return True

    return False

# Asumiendo que nlp, check_dependency y parsed_entities están definidos previamente

G = nx.Graph()

print("🕸️ Construyendo grafo basado en sintaxis...")

for item in parsed_entities:
    sentence_text = item["text"]
    entities = item["entities"]

    # Analizamos la oración con spaCy
    doc = nlp(sentence_text)

    spacy_entities = []

    # 1. Alinear entidades del NER (caracteres) con Tokens de spaCy
    for ent in entities:
        # char_span crea un Span de spaCy basado en indices de caracteres
        # alignment_mode="contract" asegura que no rompamos palabras a la mitad
        span = doc.char_span(ent["start"], ent["end"], alignment_mode="contract")

        if span:
            spacy_entities.append({
                "span": span,     # El objeto Span de spaCy
                "lemma": ent["entity_id"], # El nombre para el nodo del grafo
                "type": ent["entity_type"]
            })

    # 2. Evaluar pares posibles
    if len(spacy_entities) < 2:
        continue

    pairs = combinations(spacy_entities, 2)

    for e1, e2 in pairs:
        # Obtenemos la raíz sintáctica de la entidad (ej: en "cancer de pulmón", la raíz es "cancer")
        root1 = e1["span"].root
        root2 = e2["span"].root

        # VERIFICACIÓN SINTÁCTICA o co-ocurrencia
        add_edge = False
        weight_increment = 1

        if check_dependency(root1, root2):
            add_edge = True
            weight_increment = 2  # Mayor peso para dependencias sintácticas
        elif abs(root1.i - root2.i) < 5:  # Co-ocurrencia si raíces están cerca (menos de 5 tokens de distancia)
            add_edge = True

        if add_edge:
            source = e1["lemma"]
            target = e2["lemma"]

            # Añadir al grafo
            if G.has_edge(source, target):
                G[source][target]["weight"] += weight_increment
            else:
                G.add_edge(source, target, weight=weight_increment)

print(f"✅ Grafo construido: {G.number_of_nodes()} nodos y {G.number_of_edges()} conexiones.")


print(f"Grafo original: {G.number_of_nodes()} nodos.")

In [ ]:
betweenness = nx.betweenness_centrality(G)

# 1. Eliminar nodos aislados o con muy pocas conexiones (Ruido)
betweenness_values = list(betweenness.values())
percentile_10 = np.percentile(betweenness_values, 10)  # Percentil 10
percentile_25 = np.percentile(betweenness_values, 25)  # Percentil 25

nodes_to_remove = []
for node in G:
    degree = G.degree(node)
    bc = betweenness[node]

    # Eliminar solo si: grado BAJO Y betweenness BAJO
    if degree < 2 and bc < percentile_25:
        nodes_to_remove.append(node)

G.remove_nodes_from(set(nodes_to_remove))

print(f"Grafo filtrado ahora tiene {G.number_of_nodes()} nodos.")

# 2. Guardar GEXF para Gephi
nx.write_gexf(G, "knowledge_graph_syntax.gexf")
print("💾 Guardado archivo 'knowledge_graph_syntax.gexf'.")

# 3. Guardar lista de nodos
with open("nodos.txt", "w", encoding="utf-8") as f:
    for node in G.nodes():
        f.write(str(node) + "\n")

# 4. Visualización Rápida
if G.number_of_nodes() > 0:
    plt.figure(figsize=(16, 12))
    # Layout spring suele funcionar bien para visualizar clusters
    pos = nx.spring_layout(G, k=0.15, iterations=20)

    # Tamaño de nodos basado en grado
    d = dict(G.degree)
    node_sizes = [v * 20 for v in d.values()]

    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color="#6495ED", alpha=0.7)
    nx.draw_networkx_edges(G, pos, alpha=0.2)
    # Solo ponemos etiquetas a los nodos más importantes para no saturar
    labels = {n: n for n in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=8, font_color="black")

    plt.title("Grafo de Conocimiento (Filtrado por Sintaxis)")
    plt.axis("off")
    plt.savefig("grafo_sintactico.png", dpi=300, bbox_inches="tight")
    plt.show()
else:
    print("⚠️ El grafo quedó vacío después del filtrado. Intenta bajar la exigencia del score NER o el grado mínimo.")

In [ ]:
with open("nodos.txt", "w", encoding="utf-8") as f:
    for node in G.nodes():
        f.write(str(node) + "\n")

In [ ]:
ontologies_map = {
    "HPO (Phenotypes)": "http://purl.obolibrary.org/obo/hp.owl",
    "DoCS": "http://purl.obolibrary.org/obo/doid/translations/doid-es.owl",
    "NCIT (Cancer/Bio)": "http://purl.obolibrary.org/obo/ncit.owl",
    "HPO (Español)": "http://purl.obolibrary.org/obo/hp/hp-international.owl",
    "DOID (Enfermedades - Español)": "http://purl.obolibrary.org/obo/doid/translations/doid-es.owl"
}

# Validacion del grafo

In [ ]:
import os
import requests
import networkx as nx
from difflib import get_close_matches
from owlready2 import get_ontology, World


# ==========================================
# CLASE 1: OntologyLoader
# ==========================================
class OntologyLoader:
    def __init__(self, cache_dir="./cache_ontologies"):
        self.cache_dir = cache_dir
        if not os.path.exists(self.cache_dir):
            os.makedirs(self.cache_dir)
        self.world = World()

    def load_from_url(self, url, filename=None):
        """Descarga y carga una ontología desde una URL, usando caché local."""
        if not filename:
            filename = url.split('/')[-1]
            if not filename.endswith('.owl') and not filename.endswith('.ttl'):
                filename += '.owl'

        local_path = os.path.join(self.cache_dir, filename)

        if not os.path.exists(local_path):
            print(f"⬇️ Descargando ontología desde {url}...")
            try:
                response = requests.get(url, timeout=60, stream=True)
                response.raise_for_status()
                with open(local_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"✅ Guardado en {local_path}")
            except Exception as e:
                print(f"❌ Error descargando ontología: {e}")
                return None
        else:
            print(f"📂 Cargando ontología desde caché: {local_path}")

        try:
            onto = self.world.get_ontology(local_path).load()
            return onto
        except Exception as e:
            print(f"⚠️ Error cargando ontología con owlready2: {e}")
            return None

    def load_multiple(self, ontologies_map):
        loaded_ontologies = {}
        for name, url in ontologies_map.items():
            print(f"Procesando {name} : {url}")
            filename = url.split('/')[-1]
            if not filename.endswith('.owl') and not filename.endswith('.ttl'):
                filename += '.owl'
            onto = self.load_from_url(url, filename=filename)
            if onto:
                loaded_ontologies[name] = onto
            else:
                print(f"❌ No se pudo cargar la ontología {name}")
        return loaded_ontologies

    def get_term_labels(self, ontology):
        labels = set()
        # Patrón para detectar IDs tipo HP_12345, GO_12345, UBERON_12345
        id_pattern = re.compile(r'^[a-zA-Z]+_\d+$')

        if not ontology:
            return labels

        for c in ontology.classes():
            # Solo agregamos c.name si NO parece un ID
            if not id_pattern.match(c.name):
                labels.add(c.name.lower())

            # Agregamos etiquetas en inglés
            if hasattr(c, 'label'):
                 for l in c.label:
                    if hasattr(l, 'lang') and l.lang == 'en':
                         labels.add(str(l).lower())
                    elif not hasattr(l, 'lang'): # Fallback para strings sin idioma
                         labels.add(str(l).lower())

        return labels

    def get_relationships(self, ontology):
        """Extrae relaciones válidas (object properties) de la ontología."""
        relationships = set()
        if not ontology:
            return relationships

        for prop in ontology.object_properties():
            relationships.add(prop.name.lower())
            if hasattr(prop, 'label'):
                for l in prop.label:
                    relationships.add(str(l).lower())
        return relationships

In [ ]:
class GraphValidator:
    def __init__(self, loaded_ontologies=None):  # CAMBIO: Cambia ontology_urls por loaded_ontologies (dict)
        self.loader = OntologyLoader()  # CAMBIO: Instancia el loader una vez, pero ya no lo usas para cargar aquí
        self.known_terms = set()
        self.known_relationships = set()
        self.ontologies = []
        self.ontology_labels = {}  # NUEVO: para mapear términos a ontologías
        if loaded_ontologies:
            for name, onto in loaded_ontologies.items():  # CAMBIO: Itera sobre ontologías pre-cargadas
                if onto:
                    self.ontologies.append(onto)
                    print("⚙️ Indexando términos y relaciones...", end="\r")
                    onto_labels = self.loader.get_term_labels(onto)
                    self.known_terms.update(onto_labels)
                    self.known_relationships.update(self.loader.get_relationships(onto))
                    # NUEVO: Guardar qué términos pertenecen a esta ontología
                    onto_name = name  # CAMBIO: Usa el name del dict en lugar de extraer de URL
                    for term in onto_labels:
                        if term not in self.ontology_labels:
                            self.ontology_labels[term] = set()
                        self.ontology_labels[term].add(onto_name)
                    print(f"Estos son los known terms: {self.known_terms}")
                    print(f"✅ {len(self.known_terms)} términos, {len(self.known_relationships)} relaciones indexadas.")

    def validate_term(self, term):
        """Valida si un término existe (match exacto o difuso)."""
        term_lower = term.lower()

        if term_lower in self.known_terms:
            return {"status": "valid", "match": term_lower, "type": "exact"}

        matches = get_close_matches(term_lower, list(self.known_terms), n=1, cutoff=0.85)
        if matches:
            return {"status": "valid", "match": matches[0], "type": "fuzzy"}

        return {"status": "invalid", "match": None, "type": "none"}

    def validate_term_with_ontology(self, term):
        """Valida término y devuelve en qué ontologías aparece."""
        term_lower = term.lower()
        matching_ontologies = set()

        if term_lower in self.known_terms:
            # NUEVO: Obtener las ontologías donde aparece este término
            matching_ontologies = self.ontology_labels.get(term_lower, set())
            return {
                "status": "valid",
                "match": term_lower,
                "type": "exact",
                "ontologies": list(matching_ontologies)  # NUEVO
            }

        # Para matches difusos, también necesitamos encontrar las ontologías
        matches = get_close_matches(term_lower, list(self.known_terms), n=1, cutoff=0.85)
        if matches:
            matched_term = matches[0]
            matching_ontologies = self.ontology_labels.get(matched_term, set())
            return {
                "status": "valid",
                "match": matched_term,
                "type": "fuzzy",
                "ontologies": list(matching_ontologies)  # NUEVO
            }

        return {
            "status": "invalid",
            "match": None,
            "type": "none",
            "ontologies": []  # NUEVO
        }

    def validate_edge(self, source, target, edge_type):
        """Valida una arista: nodos + tipo de relación."""
        source_validation = self.validate_term(str(source))
        target_validation = self.validate_term(str(target))

        edge_type_lower = str(edge_type).lower()
        edge_valid = edge_type_lower in self.known_relationships

        if not edge_valid:
            matches = get_close_matches(edge_type_lower, list(self.known_relationships), n=1, cutoff=0.85)
            if matches:
                edge_valid = True
                edge_match = matches[0]
            else:
                edge_match = None
        else:
            edge_match = edge_type_lower

        return {
            "source": source_validation,
            "target": target_validation,
            "edge_type": {
                "status": "valid" if edge_valid else "invalid",
                "match": edge_match,
                "original": edge_type
            },
            "fully_valid": (
                source_validation["status"] == "valid" and
                target_validation["status"] == "valid" and
                edge_valid
            )
        }


    def validate_graph(self, graph):
        """Valida nodos Y aristas del grafo NetworkX."""
        report = {
            "total_nodes": graph.number_of_nodes(),
            "valid_nodes": 0,
            "invalid_nodes": 0,
            "node_details": {},
            "edge_report": { # Nuevo: Reporte de aristas consolidado
                "total_edges": graph.number_of_edges(),
                "valid_rels": 0, # Aristas completamente válidas
                "weak_rels": 0, # Aristas inválidas (sin match completo)
                "avg_distance": 0.0, # No calculado actualmente, placeholder
                "details": {} # Detalles individuales de cada arista
            },
            "ontology_coverage": {  # NUEVO: métricas de cobertura
                "total_unique_terms_matched": 0,
                "terms_by_ontology_count": {},
                "term_ontology_distribution": {}
            }
        }

        # NUEVO: Para calcular cobertura entre ontologías
        all_matched_terms = set()
        ontology_term_counts = {}
        term_ontology_map = {}  # Mapa término -> [ontologías]

        print(f"\n🔍 Validando {graph.number_of_nodes()} nodos...")
        for node in graph.nodes():
            result = self.validate_term_with_ontology(str(node))
            report["node_details"][node] = result

            # NUEVO: Acumular estadísticas de cobertura
            if result["status"] == "valid":
                report["valid_nodes"] += 1
                all_matched_terms.add(result["match"])

                # Contar por ontología
                for ontology in result["ontologies"]:
                    ontology_term_counts[ontology] = ontology_term_counts.get(ontology, 0) + 1

                # Mapear término a ontologías
                if result["match"] not in term_ontology_map:
                    term_ontology_map[result["match"]] = set()
                term_ontology_map[result["match"]].update(result["ontologies"])
            else:
                report["invalid_nodes"] += 1

        # NUEVO: Calcular métricas de cobertura
        report["ontology_coverage"]["total_unique_terms_matched"] = len(all_matched_terms)
        report["ontology_coverage"]["terms_by_ontology_count"] = ontology_term_counts
        report["ontology_coverage"]["term_ontology_distribution"] = {
            term: list(ontologies)
            for term, ontologies in term_ontology_map.items()
        }

        # Resto del código original para aristas...
        print(f"🔗 Validando {graph.number_of_edges()} aristas...")
        for i, (source, target, data) in enumerate(graph.edges(data=True)):
            edge_type = data.get('relation', data.get('type', 'unknown')) # Prefer 'relation' from Mistral, fallback to 'type'
            result = self.validate_edge(source, target, edge_type)
            edge_key = (source, target, edge_type)
            report["edge_report"]["details"][edge_key] = result # Almacenar en el nuevo sub-diccionario

            if result["fully_valid"]:
                report["edge_report"]["valid_rels"] += 1 # Actualizar contador en sub-diccionario
            else:
                report["edge_report"]["weak_rels"] += 1 # Actualizar contador en sub-diccionario

            if (i + 1) % 100 == 0:
                print(f"   Procesadas {i + 1}/{report['edge_report']['total_edges']} aristas...", end="\r")

        print()  # Nueva línea

        if report["total_nodes"] > 0:
            report["node_precision"] = report["valid_nodes"] / report["total_nodes"]
        else:
            report["node_precision"] = 0.0

        # Calcular la precisión de las aristas usando los nuevos contadores
        if report["edge_report"]["total_edges"] > 0:
            report["edge_precision"] = report["edge_report"]["valid_rels"] / report["edge_report"]["total_edges"]
        else:
            report["edge_precision"] = 0.0

        return report

In [ ]:
loader = OntologyLoader()
loaded_ontologies = loader.load_multiple(ontologies_map)

In [ ]:
validator = GraphValidator(loaded_ontologies=loaded_ontologies.copy())

In [ ]:
"""
if 'G' not in locals():
    print("⚠️ ¡Alerta! La variable 'G' no está definida.")
else:
    print(f"🚀 Iniciando evaluación del grafo ({G.number_of_nodes()} nodos, {G.number_of_edges()} aristas)...")
    print(f"{'Ontología':<25} | {'Nodos V/I':<12} | {'Aristas V/I':<13} | {'Prec. Nodos':<12} | {'Prec. Aristas':<14}")
    print("-" * 95)

    results_summary = []

    # NUEVO: Variables para estadísticas globales
    all_terms_in_graph = set(G.nodes())
    all_matched_terms_global = set()
    term_ontology_count_global = {}  # término -> número de ontologías donde aparece
    ontology_overlap_stats = {}  # Para cada ontología, cuántos términos únicos

    for name, url in ontologies_map.items():
        try:
            print(f"Validando la ontologia {name}")
            # CAMBIO: Pasa {name: loaded_ontologies[name]} en lugar de [url]
            report = validator.validate_graph(G)

            nodes_str = f"{report['valid_nodes']}/{report['invalid_nodes']}"
            # CORRECTED: Access edge validation metrics from 'edge_report' sub-dictionary
            edges_str = f"{report['edge_report']['valid_rels']}/{report['edge_report']['weak_rels']}"
            node_prec = report['node_precision']
            edge_prec = report['edge_precision']

            print(f"{name:<25} | {nodes_str:<12} | {edges_str:<13} | {node_prec:>10.2%} | {edge_prec:>12.2%}")

            # NUEVO: Acumular datos globales
            ontology_overlap_stats[name] = {
                'unique_terms_matched': report['ontology_coverage']['total_unique_terms_matched'],
                'details': report['ontology_coverage']
            }

            # Acumular términos coincidentes
            for term, ontologies in report['ontology_coverage']['term_ontology_distribution'].items():
                all_matched_terms_global.add(term)
                term_ontology_count_global[term] = term_ontology_count_global.get(term, 0) + len(ontologies)

            results_summary.append({
                "ontology": name,
                "node_precision": node_prec,
                "edge_precision": edge_prec,
                "details": report
            })

        except Exception as e:
            print(f"{name:<25} | ❌ ERROR: {str(e)}")

    print("-" * 95)
    print("\n✅ Benchmark Completado.")

    # NUEVO: Estadísticas de cobertura global
    print("\n📊 ESTADÍSTICAS DE COBERTURA GLOBAL")
    print("=" * 60)

    # 1. Términos únicos que coincidieron en total
    print(f"\n🔹 Términos únicos que coincidieron en AL MENOS una ontología: {len(all_matched_terms_global)}")

    # 2. Términos que NO aparecieron en NINGUNA ontología
    terms_not_found = all_terms_in_graph - all_matched_terms_global
    print(f"🔹 Términos NO encontrados en NINGUNA ontología: {len(terms_not_found)}")

    if terms_not_found:
        print("   Ejemplos (primeros 10):")
        for i, term in enumerate(list(terms_not_found)[:10]):
            print(f"   {i+1}. {term}")

    # 3. Top 10 términos con más apariciones en ontologías
    print(f"\n🏆 TOP 10 términos con más apariciones en ontologías:")
    sorted_terms = sorted(term_ontology_count_global.items(), key=lambda x: x[1], reverse=True)[:10]
    for i, (term, count) in enumerate(sorted_terms, 1):
        # Obtener en qué ontologías específicamente aparece
        ontologies_for_term = []
        for result in results_summary:
            if term in result['details']['ontology_coverage']['term_ontology_distribution']:
                ontologies_for_term.append(result['ontology'])

        print(f"   {i}. {term} ({count} ontologías)")
        print(f"      Aparece en: {', '.join(ontologies_for_term[:3])}" +
              ("..." if len(ontologies_for_term) > 3 else ""))

    # 4. Distribución por ontología
    print(f"\n📈 DISTRIBUCIÓN DE TÉRMINOS POR ONTOLOGÍA:")
    for ontology_name, stats in ontology_overlap_stats.items():
        print(f"   {ontology_name}: {stats['unique_terms_matched']} términos únicos")

    # 5. Solapamiento entre ontologías (opcional)
    if len(ontology_overlap_stats) > 1:
        print(f"\n🔄 SOLAPAMIENTO ENTRE ONTOLOGÍAS:")
        ontologies_list = list(ontology_overlap_stats.keys())
        for i in range(len(ontologies_list)):
            for j in range(i+1, len(ontologies_list)):
                onto1 = ontologies_list[i]
                onto2 = ontologies_list[j]

                # Términos de cada ontología (simplificado)
                terms1 = set(results_summary[i]['details']['ontology_coverage']['term_ontology_distribution'].keys())
                terms2 = set(results_summary[j]['details']['ontology_coverage']['term_ontology_distribution'].keys())

                common_terms = terms1.intersection(terms2)
                if common_terms:
                    print(f"   {onto1} ↔ {onto2}: {len(common_terms)} términos en común")

    if results_summary:
        best_result = max(results_summary, key=lambda x: (x['node_precision'] + x['edge_precision']) / 2)
        print(f"\n🔎 Mejor ontología: {best_result['ontology']}")
        print(f"   Precisión nodos: {best_result['node_precision']:.2%}")
        print(f"   Precisión aristas: {best_result['edge_precision']:.2%}")

        # CORRECTED: Access edge_details from 'edge_report' sub-dictionary
        invalid_edges = [(k, v) for k, v in best_result['details']['edge_report']['details'].items() if not v['fully_valid']]
        if invalid_edges:
            print(f"\n⚠️ Ejemplos de aristas inválidas (primeras 5):")
            for (s, t, et), details in invalid_edges[:5]:
                print(f"   {s} --[{et}]--> {t}")
                if details['edge_type']['status'] == 'invalid':
                    print(f"      └─ Tipo de arista '{et}' no encontrado en ontología")
  """

# Aristas faltantes

In [ ]:
import networkx as nx
from itertools import combinations
from mistralai import Mistral
import json
import time

class Graph:
    def __init__(self):
        self.graph= nx.Graph()
        self.cache_file = "cache_etiquetas.json"
        self.cache = self._cargar_cache()

    def add_edge(self, source, target, relation='cooccurs_with'):
        if self.graph.has_edge(source, target):
            self.graph[source][target]["weight"] += 1
        else:
            self.graph.add_edge(source, target, weight=1, relation=relation)

    def build_relations(self, entity_names):
        pairs = combinations(entity_names, 2)

        for source, target in pairs:
            self.add_edge(source, target, relation='cooccurs_with')

    def build_graph(self, parsed_entities):
        for item in parsed_entities:
            if "entities" in item:
                entity_names = [e["entity"].lower().strip() for e in item["entities"]]
                entity_names = list(set(entity_names))

                # Si hay menos de 2 entidades, no podemos hacer una conexión
                if len(entity_names) < 2:
                    continue

                self.build_relations(entity_names)


    def _cargar_cache(self):
        """Carga el archivo JSON de caché si existe."""
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                print(f"Aviso: No se pudo leer el caché ({e}). Se iniciará uno nuevo.")
                return {}
        return {}

    def _guardar_cache(self):
        """Guarda el estado actual del caché en el archivo JSON."""
        with open(self.cache_file, 'w', encoding='utf-8') as f:
            json.dump(self.cache, f, indent=4, ensure_ascii=False)

    def _obtener_key(self, u, v):
        """Genera una clave única para identificar la arista."""
        return f"{u}-->{v}"

    def analyze_topology(self):
        print("\n🕸️ Análisis Topológico:")
        n = self.graph.number_of_nodes()
        e = self.graph.number_of_edges()

        # 1. Densidad
        density = nx.density(self.graph)
        print(f"   • Densidad: {density:.4f}")

        # 2. Componentes Conectados (importante para saber si hay islas)
        if nx.is_directed(self.graph):
            n_components = nx.number_weakly_connected_components(self.graph)
        else:
            n_components = nx.number_connected_components(self.graph)
        print(f"   • Componentes Conectados: {n_components}")

        # 3. Grado Promedio
        avg_degree = sum(dict(self.graph.degree()).values()) / n
        print(f"   • Grado Promedio: {avg_degree:.2f}")

        # 4. Clustering (solo para no dirigidos o convertir primero)
        # Indica si los vecinos de un nodo también son vecinos entre sí (común en biología)
        try:
            avg_clustering = nx.average_clustering(self.graph.to_undirected())
            print(f"   • Coeficiente de Clustering: {avg_clustering:.4f}")
        except:
            pass

        return {"density": density, "components": n_components}

    def label_edges(self, api_key, use_cache=True, batch_size=50):
        """
        Consulta etiquetas a Mistral y las persiste en caché.
        """
        aristas_totales = list(self.graph.edges())
        aristas_a_consultar = []
        G_etiquetado = nx.DiGraph()

        # 1. Identificar qué necesitamos consultar y qué ya tenemos
        for u, v in aristas_totales:
            key = self._obtener_key(u, v)
            if use_cache and key in self.cache:
                # Si usamos caché y existe, lo añadimos directamente
                G_etiquetado.add_edge(u, v, label=self.cache[key])
            else:
                # Si no usamos caché o no existe la arista, hay que preguntar a Mistral
                aristas_a_consultar.append((u, v))

        if not aristas_a_consultar:
            print("Resultado obtenido íntegramente desde el caché.")
            return G_etiquetado

        print(f"Total: {len(aristas_totales)} | Desde Caché: {len(aristas_totales) - len(aristas_a_consultar)} | Pendientes: {len(aristas_a_consultar)}")

        # 2. Procesamiento por lotes (Batching)
        client = Mistral(api_key=api_key)
        for i in range(0, len(aristas_a_consultar), batch_size):
            batch = aristas_a_consultar[i : i + batch_size]
            num_lote = i // batch_size + 1
            print(f"Procesando lote {num_lote} de {((len(aristas_a_consultar)-1)//batch_size)+1}...")

            prompt_sistema = (
                "Eres un experto en ontologías. Devuelve un JSON con una etiqueta semántica "
                "corta para cada relación [A, B]. Formato: {'relaciones': [[A, B, 'etiqueta'], ...]}"
            )
            prompt_usuario = f"Etiqueta estas aristas: {json.dumps(batch)}"

            try:
                chat_response = client.chat.complete(
                    model="mistral-small-latest",
                    messages=[
                        {"role": "system", "content": prompt_sistema},
                        {"role": "user", "content": prompt_usuario},
                    ],
                    response_format={"type": "json_object"}
                )

                datos_lote = json.loads(chat_response.choices[0].message.content)

                # Procesar resultados del lote
                for item in datos_lote.get("relaciones", []):
                    if len(item) == 3:
                        u_res, v_res, etiqueta = item

                        # CORRECCIÓN: Calcular key_res ANTES de usarlo
                        key_res = self._obtener_key(u_res, v_res)

                        # CORRECCIÓN: Añadir al grafo final usando u_res y v_res (del item)
                        G_etiquetado.add_edge(u_res, v_res, relation=etiqueta)

                        # Actualizar caché
                        self.cache[key_res] = etiqueta

                # Persistencia inmediata tras cada lote exitoso
                self._guardar_cache()

                # Pequeña pausa para respetar límites de la API
                if i + batch_size < len(aristas_a_consultar):
                    time.sleep(1)

            except Exception as e:
                print(f"Error en el lote {num_lote}: {e}")
                continue

        print(f"Proceso finalizado. Grafo resultante con {G_etiquetado.number_of_edges()} aristas.")
        return G_etiquetado

    def run(self, parsed_entities):
        self.build_graph(parsed_entities)

In [ ]:
G_original = G.copy()

# -------------------------------------------------
# PASO 1: Instalar PyTorch Geometric (si no está)
# -------------------------------------------------
try:
    import torch
    from torch_geometric.data import Data
    print("✅ PyTorch y PyG ya instalados")
except ImportError:
    print("Instalando PyTorch Geometric...")
    !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
    import torch
    from torch_geometric.data import Data

import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dist
import numpy as np

# === Focal Loss ===
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        bce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# -------------------------------------------------
# PASO 2: Convertir NetworkX a PyTorch Geometric
# -------------------------------------------------
def prepare_gcn_data(G):
    """Convierte tu grafo NetworkX a formato PyG"""
    print("📊 Preparando datos para GCN...")

    # Mapeo de nodos a índices
    node_list = list(G.nodes())
    node_to_idx = {node: i for i, node in enumerate(node_list)}
    idx_to_node = {i: node for node, i in node_to_idx.items()}

    # Features de nodos (inicialmente aleatorios, podrías usar embeddings de BERT después)
    num_nodes = len(node_list)
    node_features = torch.randn((num_nodes, 128))

    # Edge index (formato [2, num_edges])
    edge_index = []
    for u, v in G.edges():
        edge_index.append([node_to_idx[u], node_to_idx[v]])

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Crear objeto Data de PyG
    pyg_data = Data(
        x=node_features,
        edge_index=edge_index,
        num_nodes=num_nodes
    )

    print(f"  • Nodos convertidos: {pyg_data.num_nodes}")
    print(f"  • Aristas convertidas: {pyg_data.edge_index.shape[1]}")

    return pyg_data, node_to_idx, idx_to_node

# Convertir tu grafo G
pyg_data, node_to_idx, idx_to_node = prepare_gcn_data(G)

# -------------------------------------------------
# PASO 3: Definir Modelo GCN para Link Prediction
# -------------------------------------------------
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score

class GCNLinkPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = nn.Dropout(0.3)

    def encode(self, x, edge_index):
        """Genera embeddings de nodos"""
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_label_index):
        """Predice si existe arista entre pares de nodos"""
        src = z[edge_label_index[0]]
        dst = z[edge_label_index[1]]
        return (src * dst).sum(dim=-1)  # Similaridad por producto punto

# -------------------------------------------------
# PASO 4: Dividir Aristas para Entrenamiento
# -------------------------------------------------
from torch_geometric.transforms import RandomLinkSplit

print("\n🔀 Dividiendo aristas para entrenamiento/validación/test...")

transform = RandomLinkSplit(
    num_val=0.15,    # 15% validación
    num_test=0.15,   # 15% test
    is_undirected=True,
    add_negative_train_samples=False,
    neg_sampling_ratio=1.0  # 1 negativo por cada positivo
)

train_data, val_data, test_data = transform(pyg_data)

print(f"  • Aristas entrenamiento: {train_data.edge_label_index.shape[1]}")
print(f"  • Aristas validación: {val_data.edge_label_index.shape[1]}")
print(f"  • Aristas test: {test_data.edge_label_index.shape[1]}")

# -------------------------------------------------
# PASO 5: Entrenar el Modelo (actualizado con métricas)
# -------------------------------------------------
import numpy as np
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score, f1_score

def calculate_metrics(predictions, labels, threshold=0.5):
    """Calcula precision, recall, f1 y curva PRC"""
    # Convertir probabilidades a predicciones binarias
    binary_preds = (predictions >= threshold).astype(int)

    # Métricas básicas
    precision = precision_score(labels, binary_preds, zero_division=0)
    recall = recall_score(labels, binary_preds, zero_division=0)
    f1 = f1_score(labels, binary_preds, zero_division=0)

    # Curva Precision-Recall
    precision_vals, recall_vals, thresholds = precision_recall_curve(labels, predictions)
    prc_auc = auc(recall_vals, precision_vals)

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'prc_auc': prc_auc,
        'precision_curve': precision_vals,
        'recall_curve': recall_vals,
        'thresholds': thresholds
    }

def train_gcn_model_with_metrics():
    """Entrenamiento con hard negatives + focal loss"""
    print("\n🎯 Entrenando modelo GCN con HARD NEGATIVES + FOCAL LOSS...")

    # Hiperparámetros (puedes ajustar gamma y percentile)
    EPOCHS = 100
    LEARNING_RATE = 0.01
    HIDDEN_DIM = 256
    GAMMA_FOCAL = 2.0          # típico: 2.0–3.0
    ALPHA_FOCAL = 1.0
    DEGREE_PERCENTILE = 70     # nodos con grado > percentil 70 se consideran "high degree"

    model = GCNLinkPredictor(
        in_channels=128,
        hidden_channels=HIDDEN_DIM,
        out_channels=128
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = FocalLoss(alpha=ALPHA_FOCAL, gamma=GAMMA_FOCAL)

    # === Pre-calcular nodos de alto grado para hard negatives ===
    degrees = [G_original.degree(idx_to_node[i]) for i in range(pyg_data.num_nodes)]
    high_degree_threshold = np.percentile(degrees, DEGREE_PERCENTILE)
    high_degree_nodes = torch.tensor([
        i for i in range(pyg_data.num_nodes)
        if degrees[i] > high_degree_threshold
    ], dtype=torch.long)

    # Si no hay suficientes nodos de alto grado, fallback a todos
    if len(high_degree_nodes) < 100:
        print("⚠️ Pocos nodos de alto grado → usando todos los nodos para negativos")
        high_degree_nodes = torch.arange(pyg_data.num_nodes)

    # Probabilidades sesgadas hacia nodos de alto grado
    prob = torch.zeros(pyg_data.num_nodes)
    prob[high_degree_nodes] = 1.0
    prob = prob / prob.sum()
    sampler = dist.Categorical(prob)

    best_prc_auc = 0
    best_model_state = None
    best_metrics = {}
    history = []

    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad()

        # === HARD NEGATIVE SAMPLING ===
        num_neg_samples = train_data.edge_label_index.size(1)
        neg_src = sampler.sample((num_neg_samples,))
        neg_dst = sampler.sample((num_neg_samples,))
        neg_edge_index = torch.stack([neg_src, neg_dst], dim=0)

        # Asegurar que no se generen negativos que ya existan (opcional pero recomendado)
        # Esto es ligero porque el grafo es sparse
        full_edge_index = torch.cat([train_data.edge_index, neg_edge_index], dim=1)
        # (Puedes añadir un filtro más estricto si quieres)

        # Combinar positivas y negativas
        all_edge_index = torch.cat([
            train_data.edge_label_index,
            neg_edge_index
        ], dim=1)
        all_edge_labels = torch.cat([
            torch.ones(train_data.edge_label_index.size(1)),
            torch.zeros(neg_edge_index.size(1))
        ], dim=0).to(train_data.x.device)

        # Forward pass
        z = model.encode(train_data.x, train_data.edge_index)
        out = model.decode(z, all_edge_index)
        loss = criterion(out, all_edge_labels)

        # Backward
        loss.backward()
        optimizer.step()

        # Validación cada 10 épocas
        if (epoch + 1) % 10 == 0:
            model.eval()
            with torch.no_grad():
                z_val = model.encode(val_data.x, val_data.edge_index)
                val_out = model.decode(z_val, val_data.edge_label_index)
                val_pred = torch.sigmoid(val_out).cpu().numpy()
                val_true = val_data.edge_label.cpu().numpy()

                auc_score = roc_auc_score(val_true, val_pred)
                metrics = calculate_metrics(val_pred, val_true)

                history.append({
                    'epoch': epoch + 1,
                    'loss': loss.item(),
                    'auc': auc_score,
                    'precision': metrics['precision'],
                    'recall': metrics['recall'],
                    'f1': metrics['f1'],
                    'prc_auc': metrics['prc_auc']
                })

                if metrics['prc_auc'] > best_prc_auc:
                    best_prc_auc = metrics['prc_auc']
                    best_model_state = model.state_dict().copy()
                    best_metrics = metrics.copy()

                print(f' Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | '
                      f'AUC: {auc_score:.4f} | PRC-AUC: {metrics["prc_auc"]:.4f} | '
                      f'Precision: {metrics["precision"]:.4f} | Recall: {metrics["recall"]:.4f}')

    # Resumen
    print("\n" + "="*80)
    print("📊 RESUMEN DE ENTRENAMIENTO (con hard negatives + focal loss)")
    print("="*80)
    print(f" • Mejor AUC (ROC): {history[-1]['auc']:.4f}")
    print(f" • Mejor AUC (PRC): {best_prc_auc:.4f}")
    print(f" • Mejor Precision: {best_metrics.get('precision', 0):.4f}")
    print(f" • Mejor Recall: {best_metrics.get('recall', 0):.4f}")
    print(f" • Mejor F1-Score: {best_metrics.get('f1', 0):.4f}")

    return best_model_state, best_metrics, history

# Entrenar modelo con métricas
best_model_state, best_val_metrics, training_history = train_gcn_model_with_metrics()

# -------------------------------------------------
# PASO 6: Evaluar y Predecir Nuevas Conexiones (actualizado)
# -------------------------------------------------
def evaluate_model_comprehensively(model_state, threshold=0.5):
    """Evalúa el modelo con todas las métricas"""
    print("\n📈 Evaluación completa del modelo...")

    # Cargar mejor modelo
    model = GCNLinkPredictor(128, 256, 128)
    model.load_state_dict(model_state)
    model.eval()

    with torch.no_grad():
        # Obtener embeddings de todos los nodos
        z = model.encode(pyg_data.x, pyg_data.edge_index)

        # Evaluar en conjunto de test
        test_out = model.decode(z, test_data.edge_label_index)
        test_pred_probs = torch.sigmoid(test_out).cpu().numpy()
        test_true = test_data.edge_label.cpu().numpy()

        # Calcular AUC-ROC
        test_auc = roc_auc_score(test_true, test_pred_probs)

        # Calcular todas las métricas
        test_metrics = calculate_metrics(test_pred_probs, test_true, threshold)

        # Mostrar resultados
        print(f"  • AUC-ROC (Test): {test_auc:.4f}")
        print(f"  • AUC-PRC (Test): {test_metrics['prc_auc']:.4f}")
        print(f"  • Precision (Test): {test_metrics['precision']:.4f}")
        print(f"  • Recall (Test): {test_metrics['recall']:.4f}")
        print(f"  • F1-Score (Test): {test_metrics['f1']:.4f}")

        # Evaluar también en validación para comparación
        val_out = model.decode(z, val_data.edge_label_index)
        val_pred_probs = torch.sigmoid(val_out).cpu().numpy()
        val_true = val_data.edge_label.cpu().numpy()
        val_auc = roc_auc_score(val_true, val_pred_probs)
        val_metrics = calculate_metrics(val_pred_probs, val_true, threshold)

        print(f"\n  • AUC-ROC (Val): {val_auc:.4f}")
        print(f"  • AUC-PRC (Val): {val_metrics['prc_auc']:.4f}")

        return z, test_auc, test_metrics, test_pred_probs, test_true

def predict_new_links_with_threshold(model_state, top_k=50, score_threshold=None):
    """Predice nuevas conexiones con umbral adaptativo"""
    print(f"\n🔮 Prediciendo nuevas conexiones...")

    # Evaluar primero para determinar umbral óptimo
    z, test_auc, test_metrics, _, _ = evaluate_model_comprehensively(model_state)

    # Determinar umbral automáticamente si no se proporciona
    if score_threshold is None:
        # Usar umbral que maximice F1-score en validación
        model = GCNLinkPredictor(128, 256, 128)
        model.load_state_dict(model_state)
        model.eval()

        with torch.no_grad():
            z_val = model.encode(val_data.x, val_data.edge_index)
            val_out = model.decode(z_val, val_data.edge_label_index)
            val_pred_probs = torch.sigmoid(val_out).cpu().numpy()
            val_true = val_data.edge_label.cpu().numpy()

            # Encontrar umbral óptimo
            f1_scores = []
            thresholds = np.linspace(0.1, 0.9, 50)
            for th in thresholds:
                binary_preds = (val_pred_probs >= th).astype(int)
                f1 = f1_score(val_true, binary_preds, zero_division=0)
                f1_scores.append(f1)

            optimal_idx = np.argmax(f1_scores)
            score_threshold = thresholds[optimal_idx]
            print(f"  • Umbral óptimo (max F1): {score_threshold:.4f}")

    # Predecir todas las posibles conexiones faltantes
    num_nodes = pyg_data.num_nodes
    predicted_edges = []

    # Matriz de adyacencia para verificar conexiones existentes
    existing_edges = set()
    for i in range(pyg_data.edge_index.shape[1]):
        u = pyg_data.edge_index[0, i].item()
        v = pyg_data.edge_index[1, i].item()
        existing_edges.add((u, v))
        existing_edges.add((v, u))

    # Calcular similitud para pares no conectados
    print("  • Calculando similitudes...")
    batch_size = 1000
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if (i, j) not in existing_edges:
                similarity = torch.dot(z[i], z[j]).item()
                predicted_edges.append({
                    'source': idx_to_node[i],
                    'target': idx_to_node[j],
                    'score': similarity,
                    'source_idx': i,
                    'target_idx': j
                })

    # Filtrar por umbral y ordenar
    predicted_edges = [e for e in predicted_edges if e['score'] >= score_threshold]
    predicted_edges.sort(key=lambda x: x['score'], reverse=True)

    # Tomar top_k o todas las que superen el umbral
    if top_k is not None:
        top_predictions = predicted_edges[:top_k]
    else:
        top_predictions = predicted_edges

    print(f"  • Predicciones sobre umbral: {len(predicted_edges)}")
    print(f"  • Top predicciones: {len(top_predictions)}")

    return test_auc, test_metrics, top_predictions, z, score_threshold

test_auc, test_metrics, gcn_predictions, node_embeddings, optimal_threshold = predict_new_links_with_threshold(
    best_model_state,
    top_k=500,
    score_threshold=0.5
)

print("\n📊 Obteniendo probabilidades de test para diagnóstico...")
z_final, test_auc_final, test_metrics_final, test_pred_probs, test_true = evaluate_model_comprehensively(
    best_model_state,
    threshold=optimal_threshold
)

# Actualizar variables si es necesario
test_auc = test_auc_final
test_metrics = test_metrics_final

# -------------------------------------------------
# PASO 7: Crear Grafo Enriquecido con GCN (actualizado)
# -------------------------------------------------
print("\n🔄 Integrando predicciones GCN al grafo original...")

# Crear grafo enriquecido
G_gcn = G_original.copy()
gcn_added_edges = 0

# Estadísticas de scores
scores = [p['score'] for p in gcn_predictions]
print(f"  • Rango de scores: [{min(scores):.4f}, {max(scores):.4f}]")
print(f"  • Score promedio: {np.mean(scores):.4f}")
print(f"  • Score mediana: {np.median(scores):.4f}")

# Añadir aristas predichas
for pred in gcn_predictions:
    if pred['score'] >= optimal_threshold:
        if not G_gcn.has_edge(pred['source'], pred['target']):
            G_gcn.add_edge(
                pred['source'],
                pred['target'],
                weight=float(pred['score']),
                relation='predicted_by_GCN',
                origin='GCN_prediction',
                prediction_score=float(pred['score']),
                prediction_rank=gcn_predictions.index(pred) + 1
            )
            gcn_added_edges += 1

print(f"  • Aristas añadidas por GCN: {gcn_added_edges}")
print(f"  • Grafo GCN enriquecido: {G_gcn.number_of_nodes()} nodos, {G_gcn.number_of_edges()} aristas")

# -------------------------------------------------
# PASO 8: Visualización de Métricas
# -------------------------------------------------
import matplotlib.pyplot as plt

def plot_metrics_comparison(training_history, test_metrics, val_metrics):
    """Visualiza las métricas de entrenamiento y evaluación"""
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))

    # Extraer datos del historial
    epochs = [h['epoch'] for h in training_history]
    losses = [h['loss'] for h in training_history]
    aucs = [h['auc'] for h in training_history]
    precisions = [h['precision'] for h in training_history]
    recalls = [h['recall'] for h in training_history]
    f1s = [h['f1'] for h in training_history]

    # 1. Pérdida durante entrenamiento
    axes[0, 0].plot(epochs, losses, 'b-', linewidth=2)
    axes[0, 0].set_xlabel('Época')
    axes[0, 0].set_ylabel('Pérdida')
    axes[0, 0].set_title('Pérdida durante Entrenamiento')
    axes[0, 0].grid(True, alpha=0.3)

    # 2. AUC durante entrenamiento
    axes[0, 1].plot(epochs, aucs, 'g-', linewidth=2)
    axes[0, 1].set_xlabel('Época')
    axes[0, 1].set_ylabel('AUC-ROC')
    axes[0, 1].set_title('AUC durante Entrenamiento')
    axes[0, 1].grid(True, alpha=0.3)

    # 3. Precision, Recall, F1 durante entrenamiento
    axes[0, 2].plot(epochs, precisions, 'r-', label='Precision', linewidth=2)
    axes[0, 2].plot(epochs, recalls, 'g-', label='Recall', linewidth=2)
    axes[0, 2].plot(epochs, f1s, 'b-', label='F1-Score', linewidth=2)
    axes[0, 2].set_xlabel('Época')
    axes[0, 2].set_ylabel('Valor')
    axes[0, 2].set_title('Métricas durante Entrenamiento')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)

    # 4. Curva Precision-Recall
    axes[1, 0].plot(test_metrics['recall_curve'], test_metrics['precision_curve'],
                     'b-', linewidth=2, label=f'Test (AUC={test_metrics["prc_auc"]:.3f})')
    axes[1, 0].set_xlabel('Recall')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].set_title('Curva Precision-Recall')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # 5. Comparación de Métricas Finales
    metrics_names = ['Precision', 'Recall', 'F1-Score', 'AUC-PRC']
    test_values = [
        test_metrics['precision'],
        test_metrics['recall'],
        test_metrics['f1'],
        test_metrics['prc_auc']
    ]
    val_values = [
        val_metrics['precision'],
        val_metrics['recall'],
        val_metrics['f1'],
        val_metrics['prc_auc']
    ]

    x = np.arange(len(metrics_names))
    width = 0.35
    axes[1, 1].bar(x - width/2, test_values, width, label='Test', color='skyblue')
    axes[1, 1].bar(x + width/2, val_values, width, label='Validación', color='lightcoral')
    axes[1, 1].set_xlabel('Métricas')
    axes[1, 1].set_ylabel('Valor')
    axes[1, 1].set_title('Comparación Test vs Validación')
    axes[1, 1].set_xticks(x)
    axes[1, 1].set_xticklabels(metrics_names, rotation=45)
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3, axis='y')

    # 6. Distribución de Scores de Predicción
    axes[1, 2].hist(scores, bins=20, alpha=0.7, color='purple', edgecolor='black')
    axes[1, 2].axvline(x=optimal_threshold, color='red', linestyle='--',
                       label=f'Umbral={optimal_threshold:.3f}')
    axes[1, 2].set_xlabel('Score de Predicción')
    axes[1, 2].set_ylabel('Frecuencia')
    axes[1, 2].set_title('Distribución de Scores')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('gcn_metrics_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

    return fig

# Obtener métricas de validación para comparación
model = GCNLinkPredictor(128, 256, 128)
model.load_state_dict(best_model_state)
model.eval()

with torch.no_grad():
    z_val = model.encode(val_data.x, val_data.edge_index)
    val_out = model.decode(z_val, val_data.edge_label_index)
    val_pred_probs = torch.sigmoid(val_out).cpu().numpy()
    val_true = val_data.edge_label.cpu().numpy()
    val_metrics = calculate_metrics(val_pred_probs, val_true, optimal_threshold)

# Generar visualización
print("\n📊 Generando visualización de métricas...")
fig = plot_metrics_comparison(training_history, test_metrics, val_metrics)

# -------------------------------------------------
# PASO 9: Reporte Final
# -------------------------------------------------
print("\n" + "="*80)
print("📊 REPORTE FINAL - GCN LINK PREDICTION")
print("="*80)

print(f"\n📈 MÉTRICAS DE EVALUACIÓN:")
print(f"  • AUC-ROC (Test): {test_auc:.4f}")
print(f"  • AUC-PRC (Test): {test_metrics['prc_auc']:.4f}")
print(f"  • Precision (Test): {test_metrics['precision']:.4f}")
print(f"  • Recall (Test): {test_metrics['recall']:.4f}")
print(f"  • F1-Score (Test): {test_metrics['f1']:.4f}")

print(f"\n🎯 PREDICCIONES:")
print(f"  • Umbral óptimo: {optimal_threshold:.4f}")
print(f"  • Predicciones generadas: {len(gcn_predictions)}")
print(f"  • Aristas añadidas al grafo: {gcn_added_edges}")
print(f"  • Score promedio de predicciones: {np.mean(scores):.4f}")

print(f"\n📁 GRAFO RESULTANTE:")
print(f"  • Nodos: {G_gcn.number_of_nodes()}")
print(f"  • Aristas: {G_gcn.number_of_edges()}")
print(f"  • Densidad: {nx.density(G_gcn):.6f}")

# Guardar grafo enriquecido
nx.write_gexf(G_gcn, "knowledge_graph_gcn_enriched.gexf")
print("\n💾 Grafo GCN guardado como 'knowledge_graph_gcn_enriched.gexf'")
print("💾 Gráfico de métricas guardado como 'gcn_metrics_comparison.png'")

# -------------------------------------------------
# PASO 10: Comparación con PyKEEN (si lo tienes)
# -------------------------------------------------
print("\n" + "="*80)
print("⚖️ COMPARACIÓN ENTRE MÉTODOS DE PREDICCIÓN")
print("="*80)

# Si tienes resultados de PyKEEN, puedes agregarlos aquí:
# pykeen_metrics = obtener_metricas_pykeen()  # Tu función aquí

print("\n✅ GCN Link Prediction completado con todas las métricas")

In [ ]:
if 'G_gcn' in locals():
    print("\n🔍 Validando grafo enriquecido con GCN...")
    # Usar tu validador existente
    gcn_report = validator.validate_graph(G_gcn)

In [ ]:

print(gcn_report["valid_nodes"])
print(f"Resultados validacion {gcn_report["edge_report"]}")

In [ ]:
!pip uninstall community
!pip install python-louvain

def analyze_class_imbalance(G_original):
    """Analiza el desequilibrio entre aristas existentes y posibles"""
    n = G_original.number_of_nodes()
    possible_edges = n * (n - 1) / 2 if n > 1 else 0
    existing_edges = G_original.number_of_edges()
    missing_edges = possible_edges - existing_edges

    imbalance_ratio = missing_edges / existing_edges if existing_edges > 0 else float('inf')

    print(f"📊 Análisis de Desequilibrio de Clases:")
    print(f"  • Nodos: {n}")
    print(f"  • Aristas existentes (positivas): {existing_edges}")
    print(f"  • Aristas posibles (total): {possible_edges:.0f}")
    print(f"  • Aristas faltantes (negativas): {missing_edges:.0f}")
    print(f"  • Ratio desequilibrio (neg:pos): {imbalance_ratio:.2f}:1")
    print(f"  • % Aristas positivas: {(existing_edges/possible_edges)*100:.6f}%")
    print(f"  • Densidad del grafo: {nx.density(G_original):.6f}")

    return imbalance_ratio

def check_data_leakage(G_original, train_data, test_data, idx_to_node):
    """Verifica si hay leakage en la división train/test"""
    print("\n🔍 Verificando Data Leakage...")

    # Verificar si hay nodos aislados
    isolated_nodes = [n for n in G_original.nodes() if G_original.degree(n) == 0]
    print(f"  • Nodos aislados: {len(isolated_nodes)}")

    # Verificar aristas de test que podrían ser predecibles por grado
    test_positives = 0
    easy_predictions = 0

    for i in range(test_data.edge_label_index.shape[1]):
        if test_data.edge_label[i].item() == 1:  # Arista positiva
            u = test_data.edge_label_index[0, i].item()
            v = test_data.edge_label_index[1, i].item()
            test_positives += 1

            # Verificar si ambos nodos tienen alto grado
            if G_original.degree(idx_to_node[u]) > 5 and G_original.degree(idx_to_node[v]) > 5:
                easy_predictions += 1

    if test_positives > 0:
        print(f"  • Aristas test con ambos nodos alto grado: {(easy_predictions/test_positives)*100:.1f}%")

    # Verificar solapamiento train-test
    train_edges_set = set()
    for i in range(train_data.edge_index.shape[1]):
        u = train_data.edge_index[0, i].item()
        v = train_data.edge_index[1, i].item()
        train_edges_set.add((min(u, v), max(u, v)))

    test_train_overlap = 0
    for i in range(test_data.edge_label_index.shape[1]):
        if test_data.edge_label[i].item() == 1:
            u = test_data.edge_label_index[0, i].item()
            v = test_data.edge_label_index[1, i].item()
            if (min(u, v), max(u, v)) in train_edges_set:
                test_train_overlap += 1

    if test_positives > 0:
        print(f"  • Solapamiento train-test: {(test_train_overlap/test_positives)*100:.1f}%")

    return len(isolated_nodes)

def analyze_prediction_bias(predictions, test_pred_probs, test_true, G_original, idx_to_node):
    """Analiza el sesgo en las predicciones"""
    print("\n🎯 Análisis de Sesgo Predictivo:")

    # Distribución de predicciones
    threshold = 0.5
    positive_preds = sum(test_pred_probs >= threshold)
    negative_preds = sum(test_pred_probs < threshold)

    print(f"  • Predicciones positivas (≥{threshold}): {positive_preds}")
    print(f"  • Predicciones negativas (<{threshold}): {negative_preds}")
    print(f"  • Ratio positivo/negativo: {positive_preds/max(negative_preds, 1):.3f}")

    # Calcular métricas específicas
    cm = confusion_matrix(test_true, (test_pred_probs >= threshold).astype(int))

    print(f"\n  📊 Matriz de Confusión (threshold={threshold}):")
    print(f"                    Predicho")
    print(f"                  No      Sí")
    print(f"      Real No  [TN={cm[0,0]:4d}  FP={cm[0,1]:4d}]")
    print(f"           Sí  [FN={cm[1,0]:4d}  TP={cm[1,1]:4d}]")

    # Métricas detalladas
    if cm[1,1] + cm[0,1] > 0:
        precision = cm[1,1] / (cm[1,1] + cm[0,1])
    else:
        precision = 0

    if cm[1,1] + cm[1,0] > 0:
        recall = cm[1,1] / (cm[1,1] + cm[1,0])
    else:
        recall = 0

    print(f"\n  📈 Métricas por threshold {threshold}:")
    print(f"      • Precision: {precision:.4f}")
    print(f"      • Recall: {recall:.4f}")
    print(f"      • F1-Score: {2*precision*recall/max(precision+recall, 1e-10):.4f}")

    # Análisis de scores de predicciones
    if predictions:
        scores = [p['score'] for p in predictions]
        print(f"\n  📊 Estadísticas de Scores de Predicción:")
        print(f"      • Mínimo: {min(scores):.4f}")
        print(f"      • Máximo: {max(scores):.4f}")
        print(f"      • Mediana: {np.median(scores):.4f}")
        print(f"      • Media: {np.mean(scores):.4f}")
        print(f"      • Desviación estándar: {np.std(scores):.4f}")

        # Distribución por percentiles
        percentiles = [10, 25, 50, 75, 90, 95, 99]
        print(f"      • Percentiles:")
        for p in percentiles:
            print(f"          P{p}: {np.percentile(scores, p):.4f}")

    # Análisis de correlación con grado de nodos
    print(f"\n  🔗 Correlación con Grado de Nodos:")
    degrees = []
    pred_scores = []

    for pred in predictions[:100]:  # Muestra de predicciones
        node = pred['source']
        if node in G_original:
            degree = G_original.degree(node)
            degrees.append(degree)
            pred_scores.append(pred['score'])

    if len(degrees) > 1:
        correlation = np.corrcoef(degrees, pred_scores)[0, 1]
        print(f"      • Correlación grado-score: {correlation:.4f}")

        if correlation > 0.7:
            print("      ⚠️  ALTO: El modelo puede estar prediciendo solo por grado alto")
        elif correlation > 0.4:
            print("      ℹ️  MODERADO: El modelo considera el grado pero no exclusivamente")
        else:
            print("      ✅ BAJO: El modelo no depende fuertemente del grado")

def evaluate_negative_sampling(G_original, train_data, test_data, idx_to_node):
    """Evalúa si los negativos son demasiado fáciles de identificar"""
    print("\n🔄 Evaluando Muestreo de Negativos...")

    # Calcular métricas de los negativos generados
    neg_degrees = []
    pos_degrees = []

    # Grados de nodos en aristas positivas (train)
    for i in range(train_data.edge_index.shape[1]):
        u = train_data.edge_index[0, i].item()
        v = train_data.edge_index[1, i].item()
        pos_degrees.append(G_original.degree(idx_to_node[u]))
        pos_degrees.append(G_original.degree(idx_to_node[v]))

    # Grados de nodos en aristas negativas (test)
    for i in range(test_data.edge_label_index.shape[1]):
        if test_data.edge_label[i].item() == 0:  # Arista negativa
            u = test_data.edge_label_index[0, i].item()
            v = test_data.edge_label_index[1, i].item()
            neg_degrees.append(G_original.degree(idx_to_node[u]))
            neg_degrees.append(G_original.degree(idx_to_node[v]))

    if pos_degrees and neg_degrees:
        print(f"  • Grado promedio positivos: {np.mean(pos_degrees):.2f}")
        print(f"  • Grado promedio negativos: {np.mean(neg_degrees):.2f}")
        print(f"  • Diferencia: {np.mean(pos_degrees) - np.mean(neg_degrees):.2f}")

        # Test estadístico simple
        if abs(np.mean(pos_degrees) - np.mean(neg_degrees)) > 5:
            print(f"  ⚠️  Los negativos tienen grado significativamente diferente")

    # Verificar si los negativos son triviales
    trivial_negatives = 0
    total_negatives = 0

    for i in range(test_data.edge_label_index.shape[1]):
        if test_data.edge_label[i].item() == 0:
            u = test_data.edge_label_index[0, i].item()
            v = test_data.edge_label_index[1, i].item()
            total_negatives += 1

            # Si los nodos están en componentes conexas diferentes o muy lejanos
            try:
                if not nx.has_path(G_original, idx_to_node[u], idx_to_node[v]):
                    trivial_negatives += 1
            except:
                trivial_negatives += 1

    if total_negatives > 0:
        trivial_percentage = trivial_negatives/total_negatives*100
        print(f"  • Negativos triviales (sin camino): {trivial_percentage:.1f}%")

        if trivial_percentage > 80:
            print(f"  ⚠️  La mayoría de negativos son triviales - AUC inflado")

    return trivial_percentage if total_negatives > 0 else 0

def calculate_balanced_metrics(test_pred_probs, test_true):
    """Calcula métricas en conjunto balanceado"""
    print("\n📈 Recalculo de Métricas con Balanceo:")

    # Identificar positivos y negativos
    pos_indices = np.where(test_true == 1)[0]
    neg_indices = np.where(test_true == 0)[0]

    if len(pos_indices) == 0 or len(neg_indices) == 0:
        print("  ❌ No hay suficientes muestras para balancear")
        return None, None

    # Tomar muestra balanceada (1:1)
    min_samples = min(len(pos_indices), len(neg_indices))

    balanced_pos = np.random.choice(pos_indices, min_samples, replace=False)
    balanced_neg = np.random.choice(neg_indices, min_samples, replace=False)

    balanced_indices = np.concatenate([balanced_pos, balanced_neg])
    balanced_true = test_true[balanced_indices]
    balanced_pred = test_pred_probs[balanced_indices]

    # Calcular métricas balanceadas
    balanced_auc = roc_auc_score(balanced_true, balanced_pred)

    # Calcular métricas originales para comparación
    original_auc = roc_auc_score(test_true, test_pred_probs)

    print(f"  • Muestras balanceadas: {2*min_samples} ({min_samples} positivas, {min_samples} negativas)")
    print(f"  • AUC original (desbalanceado): {original_auc:.4f}")
    print(f"  • AUC balanceado (1:1): {balanced_auc:.4f}")
    print(f"  • Diferencia: {original_auc - balanced_auc:+.4f}")

    if original_auc - balanced_auc > 0.1:
        print(f"  ⚠️  Gran diferencia: AUC inflado por desbalance")

    # Calcular Precision-Recall AUC
    precision_vals, recall_vals, _ = precision_recall_curve(test_true, test_pred_probs)
    pr_auc = auc(recall_vals, precision_vals)

    # PR AUC balanceado
    precision_vals_bal, recall_vals_bal, _ = precision_recall_curve(balanced_true, balanced_pred)
    pr_auc_bal = auc(recall_vals_bal, precision_vals_bal)

    print(f"  • PR-AUC original: {pr_auc:.4f}")
    print(f"  • PR-AUC balanceado: {pr_auc_bal:.4f}")

    return balanced_auc, pr_auc

def analyze_model_consistency(predictions, G_original, top_n=50):
    """Analiza la consistencia de las predicciones"""
    print("\n🔄 Consistencia de Predicciones:")

    if not predictions:
        print("  ❌ No hay predicciones para analizar")
        return

    # Verificar variabilidad en scores
    top_scores = [p['score'] for p in predictions[:top_n]]
    unique_scores = len(set([round(s, 3) for s in top_scores]))

    print(f"  • Scores únicos en top {top_n}: {unique_scores}")

    if unique_scores < top_n * 0.3:
        print(f"  ⚠️  Poca variabilidad en predicciones")

    # Verificar patrones en predicciones
    print(f"\n  🔎 Patrones en Top {top_n} Predicciones:")

    # Contar predicciones intra-comunidad vs inter-comunidad
    try:
        import community as community_louvain
        partition = community_louvain.best_partition(G_original)

        intra_count = 0
        inter_count = 0

        for pred in predictions[:top_n]:
            if pred['source'] in partition and pred['target'] in partition:
                if partition[pred['source']] == partition[pred['target']]:
                    intra_count += 1
                else:
                    inter_count += 1

        total = intra_count + inter_count
        if total > 0:
            print(f"      • Intra-comunidad: {intra_count} ({intra_count/total*100:.1f}%)")
            print(f"      • Inter-comunidad: {inter_count} ({inter_count/total*100:.1f}%)")
    except Exception as e:
        print(f"• No se pudo analizar comunidades (instala python-louvain): {e}")

    # Verificar si predice a nodos populares
    high_degree_nodes = [n for n in G_original.nodes() if G_original.degree(n) > 10]
    predictions_to_high_degree = 0

    for pred in predictions[:top_n]:
        if pred['source'] in high_degree_nodes or pred['target'] in high_degree_nodes:
            predictions_to_high_degree += 1

    print(f"      • Predicciones a nodos alto grado: {predictions_to_high_degree}/{top_n} ({predictions_to_high_degree/top_n*100:.1f}%)")

def comprehensive_diagnosis(G_original, G_gcn, predictions, test_pred_probs, test_true,
                           train_data, test_data, idx_to_node):
    """Diagnóstico completo del aumento artificial de AUC"""
    print("="*80)
    print("🔬 DIAGNÓSTICO COMPLETO - ¿POR QUÉ AUMENTA EL AUC ARTIFICIALMENTE?")
    print("="*80)

    # 1. Análisis de desequilibrio
    imbalance_ratio = analyze_class_imbalance(G_original)

    # 2. Verificar data leakage
    isolated_nodes = check_data_leakage(G_original, train_data, test_data, idx_to_node)

    # 3. Análisis de sesgo predictivo
    analyze_prediction_bias(predictions, test_pred_probs, test_true, G_original, idx_to_node)

    # 4. Evaluar muestreo de negativos
    trivial_percentage = evaluate_negative_sampling(G_original, train_data, test_data, idx_to_node)

    # 5. Recalcular métricas con balanceo
    balanced_auc, pr_auc = calculate_balanced_metrics(test_pred_probs, test_true)

    # 6. Analizar consistencia del modelo
    analyze_model_consistency(predictions, G_original)

    # 7. Resumen y conclusiones
    print("\n" + "="*80)
    print("📋 RESUMEN Y CONCLUSIONES")
    print("="*80)

    conclusions = []

    if imbalance_ratio > 100:
        conclusions.append("🔴 DESBALANCE EXTREMO: AUC inflado por mayoría de negativos")
    elif imbalance_ratio > 50:
        conclusions.append("🟠 DESBALANCE ALTO: Posible inflación de AUC")
    else:
        conclusions.append("🟢 DESBALANCE MODERADO: AUC probablemente realista")

    if trivial_percentage > 80:
        conclusions.append("🔴 NEGATIVOS TRIVIALES: Negativos demasiado fáciles de identificar")
    elif trivial_percentage > 50:
        conclusions.append("🟠 NEGATIVOS FÁCILES: Algunos negativos son triviales")
    else:
        conclusions.append("🟢 NEGATIVOS ADECUADOS: Muestreo de negativos correcto")

    if isolated_nodes > G_original.number_of_nodes() * 0.1:
        conclusions.append("🔴 NODOS AISLADOS: Muchos nodos sin conexiones")

    # AUC balanceado vs original
    original_auc = roc_auc_score(test_true, test_pred_probs)
    if balanced_auc and (original_auc - balanced_auc) > 0.15:
        conclusions.append(f"🔴 AUC INFLADO: Diferencia de {original_auc - balanced_auc:.3f} por desbalance")
    elif balanced_auc and (original_auc - balanced_auc) > 0.05:
        conclusions.append(f"🟠 AUC MODERADAMENTE INFLADO: Diferencia de {original_auc - balanced_auc:.3f}")

    print("\n🎯 Conclusiones principales:")
    for i, conclusion in enumerate(conclusions, 1):
        print(f"  {i}. {conclusion}")

    print("\n💡 Recomendaciones:")
    print("  1. Usar AUC-PRC en lugar de AUC-ROC para datos desbalanceados")
    print("  2. Reportar Precision@K y Recall@K para evaluaciones prácticas")
    print("  3. Implementar negative sampling más desafiante")
    print("  4. Evaluar en diferentes thresholds")
    print("  5. Usar cross-validation por comunidades")

    # Métricas prácticas
    print("\n📊 Métricas Prácticas para Link Prediction:")

    def precision_at_k(model, test_data, k=100):
        model.eval()
        with torch.no_grad():
            z = model.encode(test_data.x, test_data.edge_index)
            scores = model.decode(z, test_data.edge_label_index).cpu().numpy()
            true_labels = test_data.edge_label.cpu().numpy()
            # Ordenar por score descendente
            sorted_indices = np.argsort(scores)[::-1]
            # Tomar top k
            top_k_indices = sorted_indices[:k]
            # Calcular precisión en top k
            precision = precision_score(true_labels[top_k_indices],
                                        (scores[top_k_indices] >= 0.5).astype(int))
            return precision

    precision_results = {}

    # Calcular Precision@K
    for k in [10, 20, 50, 100]:
        prec = precision_at_k(model, test_data, k=k)
        print(f"Precision@{k}: {prec:.4f}")

    return {
        'imbalance_ratio': imbalance_ratio,
        'trivial_negatives_percentage': trivial_percentage,
        'original_auc': original_auc,
        'balanced_auc': balanced_auc,
        'pr_auc': pr_auc,
    }

# ============================================================================
# PASO 3: FUNCIÓN PARA VISUALIZAR RESULTADOS
# ============================================================================
def visualize_diagnosis_results(G_original, predictions, test_pred_probs, test_true,
                               diagnosis_results, top_k=50):
    """Genera visualizaciones para el diagnóstico"""
    print("\n📊 Generando visualizaciones...")

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # 1. Distribución de grados del grafo
    degrees = [d for n, d in G_original.degree()]
    axes[0, 0].hist(degrees, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_xlabel('Grado del nodo')
    axes[0, 0].set_ylabel('Frecuencia')
    axes[0, 0].set_title('Distribución de Grados del Grafo')
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Distribución de scores de predicción
    if predictions:
        scores = [p['score'] for p in predictions[:top_k*2]]
        axes[0, 1].hist(scores, bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
        axes[0, 1].axvline(x=0.5, color='red', linestyle='--', label='Umbral 0.5')
        axes[0, 1].set_xlabel('Score de predicción')
        axes[0, 1].set_ylabel('Frecuencia')
        axes[0, 1].set_title('Distribución de Scores de Predicción')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

    # 3. Curva Precision-Recall
    precision_vals, recall_vals, thresholds = precision_recall_curve(test_true, test_pred_probs)
    axes[0, 2].plot(recall_vals, precision_vals, 'b-', linewidth=2)
    axes[0, 2].set_xlabel('Recall')
    axes[0, 2].set_ylabel('Precision')
    axes[0, 2].set_title(f'Curva Precision-Recall (AUC={diagnosis_results.get("pr_auc", 0):.3f})')
    axes[0, 2].grid(True, alpha=0.3)

    # 4. Comparación de métricas
    metrics_to_compare = ['AUC-ROC', 'AUC-PRC']
    values = [
        diagnosis_results.get('original_auc', 0),
        diagnosis_results.get('pr_auc', 0)
    ]

    if diagnosis_results.get('balanced_auc'):
        metrics_to_compare.append('AUC-ROC Balanceado')
        values.append(diagnosis_results['balanced_auc'])

    x_pos = np.arange(len(metrics_to_compare))
    bars = axes[1, 0].bar(x_pos, values, color=['skyblue', 'lightcoral', 'lightgreen'][:len(values)])
    axes[1, 0].set_ylabel('Valor')
    axes[1, 0].set_title('Comparación de Métricas')
    axes[1, 0].set_xticks(x_pos)
    axes[1, 0].set_xticklabels(metrics_to_compare, rotation=45)
    axes[1, 0].grid(True, alpha=0.3, axis='y')

    # Añadir valores encima de las barras
    for bar, val in zip(bars, values):
        height = bar.get_height()
        axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{val:.3f}', ha='center', va='bottom')

    # 5. Precision@K
    if 'precision_at_k' in diagnosis_results:
        k_values = list(diagnosis_results['precision_at_k'].keys())
        prec_values = list(diagnosis_results['precision_at_k'].values())

        axes[1, 1].plot(k_values, prec_values, 'go-', linewidth=2, markersize=8)
        axes[1, 1].set_xlabel('K')
        axes[1, 1].set_ylabel('Precision')
        axes[1, 1].set_title('Precision@K')
        axes[1, 1].grid(True, alpha=0.3)

    # 6. Matriz de confusión (threshold=0.5)
    cm = confusion_matrix(test_true, (test_pred_probs >= 0.5).astype(int))

    im = axes[1, 2].imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    axes[1, 2].set_title('Matriz de Confusión (threshold=0.5)')

    # Añadir texto en las celdas
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            axes[1, 2].text(j, i, format(cm[i, j], 'd'),
                          ha="center", va="center",
                          color="white" if cm[i, j] > thresh else "black")

    axes[1, 2].set_ylabel('Etiqueta Real')
    axes[1, 2].set_xlabel('Etiqueta Predicha')
    axes[1, 2].set_xticks([0, 1])
    axes[1, 2].set_yticks([0, 1])
    axes[1, 2].set_xticklabels(['No', 'Sí'])
    axes[1, 2].set_yticklabels(['No', 'Sí'])

    plt.tight_layout()
    plt.savefig('diagnosis_results.png', dpi=300, bbox_inches='tight')
    plt.show()

    return fig

# ============================================================================
# PASO 4: EJECUTAR DIAGNÓSTICO COMPLETO
# ============================================================================
print("🚀 Iniciando diagnóstico completo...")

# Verificar que tenemos todos los datos necesarios
required_vars = ['G_original', 'G_gcn', 'gcn_predictions', 'test_pred_probs',
                 'test_true', 'train_data', 'test_data', 'idx_to_node']

missing_vars = []
for var in required_vars:
    if var not in locals():
        missing_vars.append(var)

if missing_vars:
    print(f"⚠️ Variables faltantes: {missing_vars}")
    print("Por favor, asegúrate de haber ejecutado el entrenamiento del modelo GCN primero.")
else:
    # Ejecutar diagnóstico
    diagnosis_results = comprehensive_diagnosis(
        G_original=G_original,
        G_gcn=G_gcn,
        predictions=gcn_predictions,
        test_pred_probs=test_pred_probs,
        test_true=test_true,
        train_data=train_data,
        test_data=test_data,
        idx_to_node=idx_to_node
    )

    # Generar visualizaciones
    fig = visualize_diagnosis_results(
        G_original=G_original,
        predictions=gcn_predictions,
        test_pred_probs=test_pred_probs,
        test_true=test_true,
        diagnosis_results=diagnosis_results,
        top_k=50
    )

    print("\n" + "="*80)
    print("✅ DIAGNÓSTICO COMPLETADO")
    print("="*80)
    print("\nLos resultados se han guardado en 'diagnosis_results.png'")
    print("Revisa las conclusiones y recomendaciones arriba.")

### Clase ExtendedGraph

In [ ]:
import pandas as pd
import networkx as nx
import torch
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from pykeen.predict import predict_target

class ExtendedGraph(Graph):
    def __init__(self, graph):
        super().__init__()
        self.graph = graph.graph.copy()
        self.predicted_edges = []

    def predict_edges(self, relacion_busqueda="relacionado_con", n_predicciones=10, epochs=100):
        """
        Entrena un modelo de Knowledge Graph Embedding y añade nuevas aristas al grafo original.
        Modifica el grafo actual y almacena las aristas predichas en self.predicted_edges.

        Args:
            relacion_busqueda (str): Tipo de relación a predecir. Por defecto "relacionado_con".
            n_predicciones (int): Número de predicciones top a considerar por nodo. Por defecto 10.
            epochs (int): Número de épocas para entrenar el modelo. Por defecto 100.
        """
        triples_list = []
        for u, v, data in self.graph.edges(data=True):
            rel = data.get('relation', 'relacionado_con')
            triples_list.append([str(u), str(rel), str(v)])

        df_triples = pd.DataFrame(triples_list, columns=['head', 'relation', 'tail'])

        tf = TriplesFactory.from_labeled_triples(triples=df_triples.values)
        training_factory, testing_factory = tf.split([0.8, 0.2], random_state=42)

        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Usando dispositivo: {device}")
        print(f"Entrenando modelo para {len(self.graph.nodes)} nodos...")

        result = pipeline(
            training=training_factory,
            testing=testing_factory,
            model='RotatE',
            epochs=epochs,
            device=device,
            random_seed=42
        )

        for u, v in self.graph.edges():
            self.graph[u][v]['origin'] = 'real'

        print("Generando predicciones de nuevas conexiones...")

        self.predicted_edges = []

        for nodo in self.graph.nodes():
            try:
                predicciones = predict_target(
                    model=result.model,
                    head=str(nodo),
                    relation=relacion_busqueda,
                    triples_factory=tf
                ).df

                top_preds = predicciones.sort_values(by='score', ascending=False).head(n_predicciones)

                for _, row in top_preds.iterrows():
                    target = row['tail_label']
                    score = row['score']

                    if not self.graph.has_edge(nodo, target) and nodo != target:
                        self.graph.add_edge(nodo, target,
                                           relation=relacion_busqueda,
                                           origin='predicha',
                                           weight=score)
                        self.predicted_edges.append({
                            'source': nodo,
                            'target': target,
                            'relation': relacion_busqueda,
                            'score': score
                        })
            except Exception as e:
                continue

        print(f"¡Proceso completado! Aristas totales: {self.graph.number_of_edges()}")
        print(f"Aristas predichas añadidas: {len(self.predicted_edges)}")

        return result

    def print_img(self, remove_outliers=True, img_name="extended_graph.png"):
        """
        Imprime el grafo enriquecido, diferenciando entre aristas reales y predichas.

        Args:
            remove_outliers (bool): Si es True, elimina nodos con grado 1 para mejor visualización.
            img_name (str): Nombre del archivo de imagen donde se guardará el grafo.
        """
        import matplotlib.pyplot as plt

        G_viz = self.graph.copy()

        if remove_outliers:
            low_degree_nodes = [node for node, degree in dict(G_viz.degree()).items() if degree <= 1]
            G_viz.remove_nodes_from(low_degree_nodes)

        pos = nx.spring_layout(G_viz, seed=42)

        plt.figure(figsize=(12, 12))

        nx.draw_networkx_nodes(G_viz, pos, node_size=300, node_color='lightblue')

        real_edges = [(u, v) for u, v, d in G_viz.edges(data=True) if d.get('origin') == 'real']
        nx.draw_networkx_edges(G_viz, pos, edgelist=real_edges, edge_color='green', label='Real', width=2)

        predicted_edges = [(u, v) for u, v, d in G_viz.edges(data=True) if d.get('origin') == 'predicha']
        nx.draw_networkx_edges(G_viz, pos, edgelist=predicted_edges, edge_color='red', style='dashed', label='Predicha', width=2)

        nx.draw_networkx_labels(G_viz, pos, font_size=10)

        plt.title("Grafo Enriquecido con Predicciones de Conexiones", fontsize=15)
        plt.legend(scatterpoints=1)
        plt.axis('off')
        plt.savefig(img_name)
        plt.show()


    def print_predictions(self):
        """
        Imprime las aristas que fueron predichas por el modelo.
        """
        if not self.predicted_edges:
            print("No hay aristas predichas almacenadas.")
            return

        print(f"\nAristas Predichas ({len(self.predicted_edges)} total):")
        print("-" * 80)
        for edge in self.predicted_edges:
            print(f"{edge['source']} --({edge['relation']}, score: {edge['score']:.4f})--> {edge['target']}")
        print("-" * 80)

    def get_predicted_edges_dataframe(self):
        """
        Retorna un DataFrame de pandas con las aristas predichas.

        Returns:
            pd.DataFrame: DataFrame con columnas 'source', 'target', 'relation', 'score'
        """
        if not self.predicted_edges:
            return pd.DataFrame(columns=['source', 'target', 'relation', 'score'])
        return pd.DataFrame(self.predicted_edges)

    def get_extended_graph(self):
        """
        Retorna un nuevo objeto Graph que contiene el grafo extendido con todas las aristas
        (originales + predichas).

        Returns:
            Graph: Nuevo objeto Graph con todas las aristas incluidas.
        """
        extended_graph_obj = Graph()
        extended_graph_obj.graph = self.graph.copy()
        return extended_graph_obj

    def export_predicted_edges_img(self, remove_outliers=True, img_name="predicted_edges_only.png"):
        """
        Exporta una imagen mostrando únicamente las aristas predichas por el modelo.

        Args:
            remove_outliers (bool): Si es True, elimina nodos con grado 1 para mejor visualización.
            img_name (str): Nombre del archivo de imagen donde se guardará el grafo.
        """
        import matplotlib.pyplot as plt

        G_predicted = nx.DiGraph()

        for u, v, data in self.graph.edges(data=True):
            if data.get('origin') == 'predicha':
                G_predicted.add_edge(u, v, **data)

        if G_predicted.number_of_edges() == 0:
            print("No hay aristas predichas para visualizar.")
            return

        if remove_outliers:
            low_degree_nodes = [node for node, degree in dict(G_predicted.degree()).items() if degree <= 1]
            G_predicted.remove_nodes_from(low_degree_nodes)

        if G_predicted.number_of_nodes() == 0:
            print("No hay nodos para visualizar después de remover outliers.")
            return

        pos = nx.spring_layout(G_predicted, seed=42, k=2, iterations=50)

        plt.figure(figsize=(14, 10))

        nx.draw_networkx_nodes(G_predicted, pos, node_size=500, node_color='lightcoral', alpha=0.9)

        edges = list(G_predicted.edges(data=True))
        weights = [d.get('weight', 0.5) for _, _, d in edges]

        nx.draw_networkx_edges(
            G_predicted,
            pos,
            edge_color=weights,
            edge_cmap=plt.cm.Reds,
            width=2,
            arrows=True,
            arrowsize=20,
            arrowstyle='->',
            connectionstyle='arc3,rad=0.1'
        )

        nx.draw_networkx_labels(G_predicted, pos, font_size=10, font_weight='bold')

        edge_labels = {(u, v): f"{d.get('weight', 0):.3f}" for u, v, d in G_predicted.edges(data=True)}
        nx.draw_networkx_edge_labels(G_predicted, pos, edge_labels, font_size=8)

        plt.title(f"Aristas Predichas por el Modelo\n({G_predicted.number_of_edges()} conexiones predichas)",
                  fontsize=16, fontweight='bold')

        sm = plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(vmin=min(weights), vmax=max(weights)))
        sm.set_array([])
        plt.colorbar(sm, label='Score de Confianza', ax=plt.gca(), shrink=0.8)

        plt.axis('off')
        plt.tight_layout()
        plt.savefig(img_name, dpi=300, bbox_inches='tight')
        plt.close()

        print(f"Imagen exportada exitosamente: {img_name}")
        print(f"Nodos en el grafo: {G_predicted.number_of_nodes()}")
        print(f"Aristas predichas: {G_predicted.number_of_edges()}")

    def export_predicted_edges_txt(self, txt_name="predicted_edges.txt", format="detailed"):
        """
        Exporta las aristas predichas a un archivo de texto.

        Args:
            txt_name (str): Nombre del archivo de texto donde se guardarán las aristas.
            format (str): Formato de exportación. Opciones:
                - "detailed": Formato detallado con toda la información
                - "simple": Formato simple (source -> target)
                - "csv": Formato CSV separado por comas
                - "tsv": Formato TSV separado por tabulaciones
        """
        if not self.predicted_edges:
            print("No hay aristas predichas para exportar.")
            return

        try:
            with open(txt_name, 'w', encoding='utf-8') as f:
                if format == "detailed":
                    f.write("=" * 80 + "\n")
                    f.write("ARISTAS PREDICHAS POR EL MODELO\n")
                    f.write("=" * 80 + "\n")
                    f.write(f"Total de aristas predichas: {len(self.predicted_edges)}\n")
                    f.write("=" * 80 + "\n\n")

                    for i, edge in enumerate(self.predicted_edges, 1):
                        f.write(f"{i}. {edge['source']} --({edge['relation']}, score: {edge['score']:.4f})--> {edge['target']}\n")

                elif format == "simple":
                    for edge in self.predicted_edges:
                        f.write(f"{edge['source']} -> {edge['target']}\n")

                elif format == "csv":
                    f.write("source,target,relation,score\n")
                    for edge in self.predicted_edges:
                        f.write(f"{edge['source']},{edge['target']},{edge['relation']},{edge['score']:.4f}\n")

                elif format == "tsv":
                    f.write("source\ttarget\trelation\tscore\n")
                    for edge in self.predicted_edges:
                        f.write(f"{edge['source']}\t{edge['target']}\t{edge['relation']}\t{edge['score']:.4f}\n")

                else:
                    print(f"Formato '{format}' no reconocido. Usando formato 'detailed'.")
                    f.write("=" * 80 + "\n")
                    f.write("ARISTAS PREDICHAS POR EL MODELO\n")
                    f.write("=" * 80 + "\n")
                    f.write(f"Total de aristas predichas: {len(self.predicted_edges)}\n")
                    f.write("=" * 80 + "\n\n")

                    for i, edge in enumerate(self.predicted_edges, 1):
                        f.write(f"{i}. {edge['source']} --({edge['relation']}, score: {edge['score']:.4f})--> {edge['target']}\n")

            print(f"Aristas predichas exportadas exitosamente a: {txt_name}")
            print(f"Formato: {format}")
            print(f"Total de aristas: {len(self.predicted_edges)}")

        except Exception as e:
            print(f"Error al exportar las aristas predichas: {e}")



In [ ]:
g = Graph()
g.graph = G.copy()
print("Esta es la evaluacion de la topologia del grafo")
print(g.analyze_topology())

top_nodes = sorted(G.degree, key=lambda x: x[1], reverse=True)[:10]

print("\n👑 Top 10 Nodos (Hubs) del Grafo:")
for node, degree in top_nodes:
    print(f"   • {node}: {degree} conexiones")

### Predecir aristas faltantes de $G$

In [ ]:
graph = Graph()
graph.graph = G.copy()
extended_graph = ExtendedGraph(graph)
result = extended_graph.predict_edges()



In [ ]:
metrics = result.metric_results.to_df()

print("\n📊 Métricas de Calidad del Modelo KGE (Vista Resumida):")
print("--------------------------------------------------")

# Filtrar métricas clave y formato 'realistic' para 'both' sides
key_metrics = [
    'mean_reciprocal_rank', # Intentar con este nombre si existe
    'inverse_harmonic_mean_rank', # Nombre común para MRR en PyKEEN
    'hits_at_1',
    'hits_at_3',
    'hits_at_10'
]

filtered_metrics = metrics[
    (metrics['Rank_type'] == 'realistic') &
    (metrics['Side'] == 'both') &
    (metrics['Metric'].isin(key_metrics))
]

# Pivotar para una mejor visualización
pivoted_metrics = filtered_metrics.pivot_table(index=['Metric'], columns='Side', values='Value')

# Ajustar el nombre de la columna para 'both' si es necesario
pivoted_metrics.columns = ['Value (both sides)']

display(pivoted_metrics)

In [ ]:
extended_graph.export_predicted_edges_img()

## Comparar G con Grafo extendido

In [ ]:
import os
import requests
import networkx as nx
from difflib import get_close_matches
from owlready2 import get_ontology, World

# 2. Verificación de variables de entrada
if 'G' not in locals() or 'extended_graph' not in locals():
    print("❌ Error: Debes tener definidos 'G' (original) y 'extended_graph' (con atributo .graph).")
else:
    G_original = G
    G_extended = extended_graph.graph

    print("\n" + "="*80)
    print("📊 COMPARACIÓN: Grafo Original vs Grafo Extendido")
    print("="*80)

    comparison_results = []

    print(f"{'Ontología':<25} | {'Aristas Originales (V/I)':<25} | {'Aristas Extendidas (V/I)':<25} | {'Mejora'}")
    print("-" * 95)

    for name, url in ontologies_map.items():
        try:
            # Validar ambos grafos
            rep_orig = validator.validate_graph(G_original)
            rep_ext = validator.validate_graph(G_extended)

            # Acceder a los datos del edge_report
            edge_report_orig = rep_orig['edge_report']
            edge_report_ext = rep_ext['edge_report']

            # Cálculo de mejora en aristas válidas
            improvement = edge_report_ext['valid_rels'] - edge_report_orig['valid_rels']
            improvement_pct = (improvement / max(1, edge_report_orig['valid_rels'])) * 100 if edge_report_orig['valid_rels'] > 0 else (100 if improvement > 0 else 0)
            symbol = "✅" if improvement > 0 else ("➖" if improvement == 0 else "❌")

            # Formateo de tabla
            orig_str = f"{edge_report_orig['valid_rels']}/{edge_report_orig['weak_rels']}"
            ext_str = f"{edge_report_ext['valid_rels']}/{edge_report_ext['weak_rels']}"
            imp_str = f"{symbol} {improvement:+d} ({improvement_pct:+.1f}%)"

            print(f"{name:<25} | {orig_str:<25} | {ext_str:<25} | {imp_str:<15}")

            comparison_results.append({
                "ontology": name,
                "original_valid_edges": edge_report_orig['valid_rels'],
                "original_invalid_edges": edge_report_orig['weak_rels'],
                "extended_valid_edges": edge_report_ext['valid_rels'],
                "extended_invalid_edges": edge_report_ext['weak_rels'],
                "improvement_absolute": improvement,
                "improvement_percentage": improvement_pct
            })

        except Exception as e:
            print(f"{name:<25} | ❌ ERROR: {str(e)}")

    print("-" * 95)
    # Análisis detallado de las aristas predichas
    if comparison_results:
        # Encontrar la ontología con la mayor mejora en aristas válidas
        best_improvement = -float('inf')
        best_ontology_name = None
        best_ontology_report = None

        for res in comparison_results:
            if res['improvement_absolute'] > best_improvement:
                best_improvement = res['improvement_absolute']
                best_ontology_name = res['ontology']
                # Need to re-validate the extended graph with this specific validator for details
                # This is a bit inefficient but ensures we use the correct ontology for details
                current_validator = GraphValidator(loaded_ontologies={name: loaded_ontologies.get(name)})
                best_ontology_report = current_validator.validate_graph(G_extended)

        if best_ontology_name:
            print(f"\n🏆 Mayor mejora en aristas válidas: {best_ontology_name}")

        # Obtener aristas nuevas (solo las que son predichas y añadidas al grafo extendido)
        new_edges_in_extended = []
        for u, v, data in G_extended.edges(data=True):
            if data.get('origin') == 'predicha': # Asumiendo que 'origin' es una propiedad que marca las aristas predichas
                new_edges_in_extended.append((u, v, data.get('relation', 'unknown')))

        if new_edges_in_extended:
            strong_new_edges_count = 0
            if best_ontology_report: # Use the validator from the best ontology for detailed check
                # Extract the validator instance from the context that produced best_ontology_report
                # This is a bit indirect, but we need an actual validator object
                # Re-initializing is simpler for this example.
                detailed_validator = GraphValidator(ontology_urls=[ontologies_map[best_ontology_name]])

                for u, v, rel in new_edges_in_extended:
                    edge_validation_result = detailed_validator.validate_edge(u, v, rel)
                    if edge_validation_result['fully_valid']:
                        strong_new_edges_count += 1

            print(f"\n🔍 Análisis de {len(new_edges_in_extended)} aristas predichas añadidas:")
            print(f"   • Validadas ontológicamente (por {best_ontology_name}): {strong_new_edges_count} ({(strong_new_edges_count/len(new_edges_in_extended))*100:.1f}%)")
        else:
            print("\n🔍 No se encontraron aristas predichas añadidas al grafo extendido para analizar.")

# Etiquetar aristas

In [ ]:
# Hazte tu api key de mistral en https://admin.mistral.ai/organization/api-keys para etiquetar las aristas
api_key = "2e7t2KjNXDm6V3eB1Zlr9U0W4zJkUNxU"

if api_key == "api_key":
    raise Exception("Genera tu api_key de mistral")

In [ ]:
labeled_graph = Graph()
labeled_graph.graph = G
labeled_graph.label_edges(api_key, use_cache=False)
labeled_graph = ExtendedGraph(labeled_graph)
labeled_graph.predict_edges()

### Evaluar grafo etiquetado

In [ ]:
import os
import requests
import networkx as nx
from difflib import get_close_matches
from owlready2 import get_ontology, World

# 2. Verificación de variables de entrada
if 'G' not in locals() or 'extended_graph' not in locals():
    print("❌ Error: Debes tener definidos 'G' (original) y 'extended_graph' (con atributo .graph).")
else:
    G_original = G
    G_extended = labeled_graph.graph

    print("\n" + "="*80)
    print("📊 COMPARACIÓN: Grafo Original vs Grafo Extendido")
    print("="*80)

    comparison_results = []

    print(f"{'Ontología':<25} | {'Aristas Originales (V/I)':<25} | {'Aristas Extendidas (V/I)':<25} | {'Mejora'}")
    print("-" * 95)

    for name, url in ontologies_map.items():
        try:
            # Validar ambos grafos
            rep_orig = validator.validate_graph(G_original)
            rep_ext = validator.validate_graph(G_extended)

            # Acceder a los datos del edge_report
            edge_report_orig = rep_orig['edge_report']
            edge_report_ext = rep_ext['edge_report']

            # Cálculo de mejora en aristas válidas
            improvement = edge_report_ext['valid_rels'] - edge_report_orig['valid_rels']
            improvement_pct = (improvement / max(1, edge_report_orig['valid_rels'])) * 100 if edge_report_orig['valid_rels'] > 0 else (100 if improvement > 0 else 0)
            symbol = "✅" if improvement > 0 else ("➖" if improvement == 0 else "❌")

            # Formateo de tabla
            orig_str = f"{edge_report_orig['valid_rels']}/{edge_report_orig['weak_rels']}"
            ext_str = f"{edge_report_ext['valid_rels']}/{edge_report_ext['weak_rels']}"
            imp_str = f"{symbol} {improvement:+d} ({improvement_pct:+.1f}%)"

            print(f"{name:<25} | {orig_str:<25} | {ext_str:<25} | {imp_str:<15}")

            comparison_results.append({
                "ontology": name,
                "original_valid_edges": edge_report_orig['valid_rels'],
                "original_invalid_edges": edge_report_orig['weak_rels'],
                "extended_valid_edges": edge_report_ext['valid_rels'],
                "extended_invalid_edges": edge_report_ext['weak_rels'],
                "improvement_absolute": improvement,
                "improvement_percentage": improvement_pct
            })

        except Exception as e:
            print(f"{name:<25} | ❌ ERROR: {str(e)}")

    print("-" * 95)
    # Análisis detallado de las aristas predichas
    if comparison_results:
        # Encontrar la ontología con la mayor mejora en aristas válidas
        best_improvement = -float('inf')
        best_ontology_name = None
        best_ontology_report = None

        for res in comparison_results:
            if res['improvement_absolute'] > best_improvement:
                best_improvement = res['improvement_absolute']
                best_ontology_name = res['ontology']
                # Need to re-validate the extended graph with this specific validator for details
                # This is a bit inefficient but ensures we use the correct ontology for details
                current_validator = GraphValidator(ontology_urls=[ontologies_map[res['ontology']]])
                best_ontology_report = current_validator.validate_graph(G_extended)

        if best_ontology_name:
            print(f"\n🏆 Mayor mejora en aristas válidas: {best_ontology_name}")

        # Obtener aristas nuevas (solo las que son predichas y añadidas al grafo extendido)
        new_edges_in_extended = []
        for u, v, data in G_extended.edges(data=True):
            if data.get('origin') == 'predicha': # Asumiendo que 'origin' es una propiedad que marca las aristas predichas
                new_edges_in_extended.append((u, v, data.get('relation', 'unknown')))

        if new_edges_in_extended:
            strong_new_edges_count = 0
            if best_ontology_report: # Use the validator from the best ontology for detailed check
                detailed_validator = GraphValidator(ontology_urls=[ontologies_map[best_ontology_name]])
                for u, v, rel in new_edges_in_extended:
                    edge_validation_result = detailed_validator.validate_edge(u, v, rel)
                    if edge_validation_result['fully_valid']:
                        strong_new_edges_count += 1

            print(f"\n🔍 Análisis de {len(new_edges_in_extended)} aristas predichas añadidas:")
            print(f"   • Validadas ontológicamente (por {best_ontology_name}): {strong_new_edges_count} ({(strong_new_edges_count/len(new_edges_in_extended))*100:.1f}%)")
        else:
            print("\n🔍 No se encontraron aristas predichas añadidas al grafo extendido para analizar.")